## Some data cleaning: lobbying

We now tidy up the lobbying data. Here are a few of the initial goals:
1. Identify which lobbying reports are relevant, and which we should remove from our consideration.
2. Column hygiene: remove unnecessary columns so as to de-clutter, and break up columns that contain dictionaries into individual columns.
3. Row hygiene: identify and drop duplicate filings.
4. Incorporate information from amendment filings into the original filings they amended, so that amendments can be ignored.
5. Deal with `incomes` and `expenses`.

We import the necessary packages here.

In [6]:
import numpy as np
import pandas as pd
import datetime
import ast

This is just for some basic text processing - it's used in (5).

In [2]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from string import punctuation as punctuation

## (1) Which filing codes can we ignore?

Let's look at a few codes. Recall that they are of the following form (omitting those starting with M and Y - midyear and annual reports - since there are none such in this time period).

In [1]:
filing_types=["RR","RA","Q1","Q1Y","1T","1TY","1A","1AY","1@","1@Y","Q2","Q2Y","2T","2TY","2A","2AY","2@","2@Y","Q3","Q3Y","3T","3TY","3A","3AY","3@","3@Y","Q4","Q4Y","4T","4TY","4A","4AY","4@","4@Y"]


|Code | Meaning| 
------|-------
|R | Registration |
|A |  Amendment |
|Q | Quarter |
|Y | No activity |
|T | Termination (lobbying for a client has finished) |
|@ | Termination amendment |

In the end, registrations and amendments to registrations (RR and RA) will not relevant to us, since there is no report of income or expenses on the registration form (this form is called form "LD-1" by the office of the Senate). See, for example, https://lda.senate.gov/filings/public/filing/286588cd-7cfd-4a82-a4a0-c5bd9265f868/print/.

It is worth noting that lobbying entities report `income` if they are lobbying on behalf of a client, and report `expenses` if they are lobbying on their own behalf (many companies employ in-house lobbyists).

For a quick sanity check, let's see if we ever have a report (in 2023 Q2) indicating both expenses and income.

In [3]:
df=pd.read_csv("../../lobbying-local-data/LDA_data/Filings_2023/filings_2023_Q2.csv")

In [306]:
df["expenses"]=df["expenses"].fillna(0)
df["income"]=df["income"].fillna(0)
df.loc[(df.income!=0) & (df.expenses!=0)]

,url,filing_uuid,filing_type,filing_type_display,filing_year,filing_period,filing_period_display,filing_document_url,filing_document_content_type,income,...,registrant_different_address,registrant_city,registrant_state,registrant_zip,registrant,client,lobbying_activities,conviction_disclosures,foreign_entities,affiliated_organizations


Good- this means we are safe to fill in missing values in these columns with zeros when the other has a non-zero value - basically, you are either reporting income or expenses.

 When both columns are zero, we should not just set the value to be zero- remember that income under $5000 is rounded down and reported to be zero (and similarly for expenses) and we should probably consider which column is relevant.

In [6]:
df["expenses"]=df["expenses"].fillna(0)
df.loc[df.expenses!=0]

,url,filing_uuid,filing_type,filing_year,income,expenses,expenses_method,termination_date,lobbying_activities,conviction_disclosures,foreign_entities,affiliated_organizations,reg,reg_id,cl,cl_id,issue_codes
4,https://lda.senate.gov/api/v1/filings/e4e51f25...,e4e51f25-3310-4626-9fb1-1b0adbc11460,Q2,2023,NaN,10000.0,A,NaN,"[{'general_issue_code': 'MAR', 'general_issue_...",[],[],[],OFFSHORE MARINE SERVICE ASSOCIATION,320771,OFFSHORE MARINE SERVICE ASSOCIATION,186602,['MAR']
8,https://lda.senate.gov/api/v1/filings/2126cffe...,2126cffe-6bba-40e7-9a1d-6d25ec1aa0ee,Q2,2023,NaN,160000.0,A,NaN,"[{'general_issue_code': 'GOV', 'general_issue_...",[],[],[],OWENS CORNING,30427,OWENS CORNING,135817,"['GOV', 'HOU', 'TRD', 'ENV', 'TRA', 'ENG']"
9,https://lda.senate.gov/api/v1/filings/a8669762...,a8669762-aa16-4ea5-9b7f-929e803d4e04,Q2,2023,NaN,80000.0,A,NaN,"[{'general_issue_code': 'HCR', 'general_issue_...",[],[],[],HOSPICE COMPASSUS,401103527,HOSPICE COMPASSUS,199389,"['HCR', 'MMM']"
10,https://lda.senate.gov/api/v1/filings/07504ff3...,07504ff3-5feb-4003-a8a6-6ffd33e0cc1a,Q2,2023,NaN,8950.0,A,NaN,"[{'general_issue_code': 'DEF', 'general_issue_...",[],[],[],THE NATIONAL COALITION PARTY OF CANADA (NCPC),401105862,THE NATIONAL COALITION PARTY OF CANADA (NCPC),210392,"['DEF', 'INT']"
12,https://lda.senate.gov/api/v1/filings/b8cd1bc0...,b8cd1bc0-6e62-45e0-a693-8fcae22ff5e8,Q2,2023,NaN,5500.0,C,NaN,"[{'general_issue_code': 'MED', 'general_issue_...",[],[],[],INTERNATIONAL CHIROPRACTORS ASSOCIATION,401105988,INTERNATIONAL CHIROPRACTORS ASSOCIATION,209303,"['MED', 'MMM', 'GOV', 'VET', 'HCR']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18331,https://lda.senate.gov/api/v1/filings/4e60ba4b...,4e60ba4b-a6c1-46e0-8fcc-cb3866058dae,Q2,2023,NaN,200000.0,A,NaN,"[{'general_issue_code': 'BUD', 'general_issue_...",[],[],[],NURSE-FAMILY PARTNERSHIP COLORADO,400275297,NURSE-FAMILY PARTNERSHIP COLORADO,188794,['BUD']
18337,https://lda.senate.gov/api/v1/filings/6c52acc6...,6c52acc6-7879-495a-bb36-fad8f62caa01,Q2,2023,NaN,53000.0,A,NaN,"[{'general_issue_code': 'HCR', 'general_issue_...",[],[],[],NATIONAL ATHLETIC TRAINERS' ASSOCIATION,400283359,NATIONAL ATHLETIC TRAINERS' ASSOCIATION,188898,['HCR']
18346,https://lda.senate.gov/api/v1/filings/59b3e8d8...,59b3e8d8-81ee-4639-8fa0-330ca5188ee0,Q2,2023,NaN,30000.0,A,NaN,"[{'general_issue_code': 'ROD', 'general_issue_...",[],[],[],TEXTILE RENTAL SERVICES ASSOCIATION,37937,TEXTILE RENTAL SERVICES ASSOCIATION,146479,"['ROD', 'LBR', 'DIS', 'CSP', 'APP', 'CAW']"
18351,https://lda.senate.gov/api/v1/filings/f629416e...,f629416e-cd3b-4bb0-95f0-1539e7940bdc,Q2,2023,NaN,987181.0,C,NaN,"[{'general_issue_code': 'ENV', 'general_issue_...",[],[],[],THE AMERICAN COUNCIL OF ENGINEERING COMPANIES,2046,AMERICAN COUNCIL OF ENGINEERING COMPANIES,102678,"['ENV', 'LBR', 'FOR', 'FIN', 'TRA', 'TOR', 'TA..."


In [39]:
df.loc[(df.income==0) & (df.expenses==0)]

,url,filing_uuid,filing_type,filing_type_display,filing_year,filing_period,filing_period_display,filing_document_url,filing_document_content_type,income,...,registrant_different_address,registrant_city,registrant_state,registrant_zip,registrant,client,lobbying_activities,conviction_disclosures,foreign_entities,affiliated_organizations
5,https://lda.senate.gov/api/v1/filings/d38e586c...,d38e586c-0e64-4358-9be7-cb412e5f5343,Q2,2nd Quarter - Report,2023,second_quarter,2nd Quarter (Apr 1 - June 30),https://lda.senate.gov/filings/public/filing/d...,text/html,0.0,...,False,WASHINGTON,DC,20005.0,"{'id': 7394, 'url': 'https://lda.senate.gov/ap...","{'id': 110733, 'url': 'https://lda.senate.gov/...","[{'general_issue_code': 'SMB', 'general_issue_...",[],[],[]
7,https://lda.senate.gov/api/v1/filings/fa3ad1c5...,fa3ad1c5-54bd-4b15-84f6-0ff4cef83f30,Q2,2nd Quarter - Report,2023,second_quarter,2nd Quarter (Apr 1 - June 30),https://lda.senate.gov/filings/public/filing/f...,text/html,0.0,...,False,Colorado Springs,CO,80920.0,"{'id': 401107290, 'url': 'https://lda.senate.g...","{'id': 54235, 'url': 'https://lda.senate.gov/a...","[{'general_issue_code': 'HCR', 'general_issue_...",[],[],[]
22,https://lda.senate.gov/api/v1/filings/d76c9559...,d76c9559-80b4-467e-b9f1-92927c8e370f,Q2,2nd Quarter - Report,2023,second_quarter,2nd Quarter (Apr 1 - June 30),https://lda.senate.gov/filings/public/filing/d...,text/html,0.0,...,False,Washington,DC,20006.0,"{'id': 21893, 'url': 'https://lda.senate.gov/a...","{'id': 128835, 'url': 'https://lda.senate.gov/...","[{'general_issue_code': 'TAX', 'general_issue_...",[],[],[]
23,https://lda.senate.gov/api/v1/filings/e8d79e69...,e8d79e69-6fa5-4c9c-bb81-933a3eec60c7,Q2,2nd Quarter - Report,2023,second_quarter,2nd Quarter (Apr 1 - June 30),https://lda.senate.gov/filings/public/filing/e...,text/html,0.0,...,False,London,NaN,NaN,"{'id': 401106703, 'url': 'https://lda.senate.g...","{'id': 51798, 'url': 'https://lda.senate.gov/a...","[{'general_issue_code': 'NAT', 'general_issue_...",[],[],[]
24,https://lda.senate.gov/api/v1/filings/0c0c654a...,0c0c654a-67ac-4ca1-8bfd-dda626aa7868,Q2,2nd Quarter - Report,2023,second_quarter,2nd Quarter (Apr 1 - June 30),https://lda.senate.gov/filings/public/filing/0...,text/html,0.0,...,False,Santa Monica,CA,90404.0,"{'id': 401105600, 'url': 'https://lda.senate.g...","{'id': 210972, 'url': 'https://lda.senate.gov/...","[{'general_issue_code': 'LBR', 'general_issue_...",[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18341,https://lda.senate.gov/api/v1/filings/03e2502d...,03e2502d-c1fd-47bb-887c-481807daea3b,Q2,2nd Quarter - Report,2023,second_quarter,2nd Quarter (Apr 1 - June 30),https://lda.senate.gov/filings/public/filing/0...,text/html,0.0,...,False,Washington,DC,20044.0,"{'id': 401104542, 'url': 'https://lda.senate.g...","{'id': 202005, 'url': 'https://lda.senate.gov/...","[{'general_issue_code': 'TRU', 'general_issue_...",[],[],[]
18343,https://lda.senate.gov/api/v1/filings/002e6ff1...,002e6ff1-1657-4e06-a89c-b3cd8722fa5a,Q2,2nd Quarter - Report,2023,second_quarter,2nd Quarter (Apr 1 - June 30),https://lda.senate.gov/filings/public/filing/0...,text/html,0.0,...,False,Washington,DC,20001.0,"{'id': 33512, 'url': 'https://lda.senate.gov/a...","{'id': 141839, 'url': 'https://lda.senate.gov/...","[{'general_issue_code': 'AER', 'general_issue_...",[],[],[]
18344,https://lda.senate.gov/api/v1/filings/306c5126...,306c5126-3096-4297-a104-ad92fcac343d,Q2,2nd Quarter - Report,2023,second_quarter,2nd Quarter (Apr 1 - June 30),https://lda.senate.gov/filings/public/filing/3...,text/html,0.0,...,False,Washington,DC,20001.0,"{'id': 33512, 'url': 'https://lda.senate.gov/a...","{'id': 141854, 'url': 'https://lda.senate.gov/...","[{'general_issue_code': 'DEF', 'general_issue_...",[],[],[]
18349,https://lda.senate.gov/api/v1/filings/1b18d803...,1b18d803-5b7b-47be-a0d4-8c7d78fa1174,Q2,2nd Quarter - Report,2023,second_quarter,2nd Quarter (Apr 1 - June 30),

In [308]:
df["dt_posted"]

0        2023-04-17T13:52:00-04:00
1        2023-05-01T13:58:12-04:00
2        2023-05-04T11:20:38-04:00
3        2023-05-09T11:09:15-04:00
4        2023-05-12T17:03:04-04:00
                   ...            
18357    2024-02-29T13:42:10-05:00
18358    2024-02-29T17:25:48-05:00
18359    2024-02-29T23:56:20-05:00
18360    2024-03-03T20:40:18-05:00
18361    2024-03-04T16:35:38-05:00
Name: dt_posted, Length: 18362, dtype: object

Note all of these dataframes are ordered by `dt_posted`. This will be important to us later when dealing with duplicates.

Let's talk about filings involving the "no-activity" code Y. While it is tempting to toss such filings, it seems very possible that firms can report income or expenses when filing a Y code. This is definitely somewhat disturbing, but maybe this corresponds to being paid to do nothing (or paid just to keep a firm on retainer)?

In [57]:
lobbying_2023_Q1Y=pd.read_csv("../../lobbying-local-data/LDA_data/Filings_2023/filings_2023_Q1Y.csv")

In [61]:
lobbying_2023_Q1Y["expenses"]=lobbying_2023_Q1Y["expenses"].fillna(0)
lobbying_2023_Q1Y["income"]=lobbying_2023_Q1Y["income"].fillna(0)

In [63]:
lobbying_2023_Q1Y.loc[(lobbying_2023_Q1Y["expenses"]!=0) | (lobbying_2023_Q1Y["income"]!=0)]

,url,filing_uuid,filing_type,filing_type_display,filing_year,filing_period,filing_period_display,filing_document_url,filing_document_content_type,income,...,registrant_different_address,registrant_city,registrant_state,registrant_zip,registrant,client,lobbying_activities,conviction_disclosures,foreign_entities,affiliated_organizations
4,https://lda.senate.gov/api/v1/filings/2f3f7ad4...,2f3f7ad4-aa65-4272-9748-cd6b2d486375,Q1Y,1st Quarter - Report (No Activity),2023,first_quarter,1st Quarter (Jan 1 - Mar 31),https://lda.senate.gov/filings/public/filing/2...,text/html,10000.0,...,False,Washington,DC,20001.0,"{'id': 401106013, 'url': 'https://lda.senate.g...","{'id': 54273, 'url': 'https://lda.senate.gov/a...",[],[],[],[]
27,https://lda.senate.gov/api/v1/filings/d18f7814...,d18f7814-89bb-43d2-b069-2ec862e55fed,Q1Y,1st Quarter - Report (No Activity),2023,first_quarter,1st Quarter (Jan 1 - Mar 31),https://lda.senate.gov/filings/public/filing/d...,text/html,0.0,...,True,Sperryvile,VA,22740.0,"{'id': 401103509, 'url': 'https://lda.senate.g...","{'id': 199224, 'url': 'https://lda.senate.gov/...",[],[],[],[]
28,https://lda.senate.gov/api/v1/filings/0afef369...,0afef369-ebe1-442e-9f82-2b123da38939,Q1Y,1st Quarter - Report (No Activity),2023,first_quarter,1st Quarter (Jan 1 - Mar 31),https://lda.senate.gov/filings/public/filing/0...,text/html,15000.0,...,False,Centreville,VA,20122.0,"{'id': 286872, 'url': 'https://lda.senate.gov/...","{'id': 176548, 'url': 'https://lda.senate.gov/...",[],[],[],[]
32,https://lda.senate.gov/api/v1/filings/22cb9607...,22cb9607-85d5-48a0-a834-a593e69441f8,Q1Y,1st Quarter - Report (No Activity),2023,first_quarter,1st Quarter (Jan 1 - Mar 31),https://lda.senate.gov/filings/public/filing/2...,text/html,30000.0,...,False,Dayton,OH,45440.0,"{'id': 401103216, 'url': 'https://lda.senate.g...","{'id': 205208, 'url': 'https://lda.senate.gov/...",[],[],[],[]
38,https://lda.senate.gov/api/v1/filings/3e0840b3...,3e0840b3-b361-43fb-8a18-31494f879e76,Q1Y,1st Quarter - Report (No Activity),2023,first_quarter,1st Quarter (Jan 1 - Mar 31),https://lda.senate.gov/filings/public/filing/3...,text/html,30000.0,...,True,Washington,DC,20037.0,"{'id': 401036920, 'url': 'https://lda.senate.g...","{'id': 208525, 'url': 'https://lda.senate.gov/...",[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2416,https://lda.senate.gov/api/v1/filings/a8d42fc9...,a8d42fc9-621b-4cf0-81fa-d2248f727d8f,Q1Y,1st Quarter - Report (No Activity),2023,first_quarter,1st Quarter (Jan 1 - Mar 31),https://lda.senate.gov/filings/public/filing/a...,text/html,12000.0,...,False,Valencia,CA,91355.0,"{'id': 401103948, 'url': 'https://lda.senate.g...","{'id': 200458, 'url': 'https://lda.senate.gov/...",[],[],[],[]
2417,https://lda.senate.gov/api/v1/filings/b10e92f1...,b10e92f1-f075-455c-aceb-f826d020b561,Q1Y,1st Quarter - Report (No Activity),2023,first_quarter,1st Quarter (Jan 1 - Mar 31),https://lda.senate.gov/filings/public/filing/b...,text/html,20000.0,...,False,Washington,DC,20005.0,"{'id': 283696, 'url': 'https://lda.senate.gov/...","{'id': 56600, 'url': 'https://lda.senate.gov/a...",[],[],[],[]
2427,https://lda.senate.gov/api/v1/filings/9678095e...,9678095e-ad4b-4c7e-a068-500641560c29,Q1Y,1st Quarter - Report (No Activity),2023,first_quarter,1st Quarter (Jan 1 - Mar 31),https://lda.senate.gov/filings/public/filing/9...,text/html,40000.0,...,False,Washington,DC,20002.0,"{'id': 30837, 'url': 'https://lda.senate.gov/a...","{'id': 136234, 'url': 'https://lda.senate.gov/...",[],[],[],[]
2433,https://lda.senate.gov/api/v1/filings/9e0a0976...,9e0a0976-a2f1-4f66-9206-499d593a1c96,Q1Y,1st Quarter - Report (No Activity),2023,first_quarter,1st Quarter (Jan 1 - Mar 31),https://lda.senate.gov/filings/public/filing/9...,text/html,40000.0,...,False,Washington,DC,20020.0,"{'id': 401105126, 'url': 'https://lda.senate.g...","{'id': 211572, 'url': 'https://lda.senate.gov/...",[],[],[],[]


So yeah, that can happen. This definitely is not what I though "no activity" would mean...

 Keep in mind too that having missing entries in both income and expenses doesn't necessarily imply that there was no lobbying that quarter - those columns have entries representing a range, so it really means something like less than $m$, where $m$ is the minimum reporting value.

OK, this has been a little all over the place. But here's our first conclusion: we will definitely omit the R (registration) codes. That leaves:

In [6]:
relevant_filing_types=["Q1","Q1Y","1T","1TY","1A","1AY","1@","1@Y","Q2","Q2Y","2T","2TY","2A","2AY","2@","2@Y","Q3","Q3Y","3T","3TY","3A","3AY","3@","3@Y","Q4","Q4Y","4T","4TY","4A","4AY","4@","4@Y"]

I am also going to standardize these codes so that the unnecessary prefix Q is removed. That leaves us with the codes below.

In [7]:
stand_filing_types=["1","1Y","1T","1TY","1A","1AY","1@","1@Y","2","2Y","2T","2TY","2A","2AY","2@","2@Y","3","3Y","3T","3TY","3A","3AY","3@","3@Y","4","4Y","4T","4TY","4A","4AY","4@","4@Y"]

The dictionary relating them is:

In [8]:
rel_2_stand=dict(zip(relevant_filing_types, stand_filing_types))

In [35]:
stand_2_rel=dict(zip(stand_filing_types, relevant_filing_types))

When I export files again, it will be with these standardized ones.

## (2) Column hygiene

Let's look at those 2023 Q2 filings again.

In [129]:
df=pd.read_csv("../../lobbying-local-data/LDA_data/Filings_2023/filings_2023_Q2.csv")


Just to see what the columns are, and what a typical entry may look like, here's the 57th row of one of these dataframes, printed out vertically.

In [5]:
df.iloc[57]

url                             https://lda.senate.gov/api/v1/filings/e7af16b4...
filing_uuid                                  e7af16b4-d06b-4230-bc30-05d781846aea
filing_type                                                                    Q2
filing_type_display                                          2nd Quarter - Report
filing_year                                                                  2023
filing_period                                                      second_quarter
filing_period_display                               2nd Quarter (Apr 1 - June 30)
filing_document_url             https://lda.senate.gov/filings/public/filing/e...
filing_document_content_type                                            text/html
income                                                                    60000.0
expenses                                                                      NaN
expenses_method                                                               NaN
expenses_method_

A lot of these columns seem to be unimportant. At a first glance, I don't think we will ever use: `["url","filing_type_display", "filing_period", "filing_period_display", "filing_document_url", "filing_document_content_type", "expenses_method_display","posted_by_name","termination_date, "registrant_country", "registrant_ppb_country", "registrant_address_1", "registrant_address_2", "registrant_different_address", "registrant_city", "registrant_state", "registrant_zip"]`. Keeping the uuid is useful if we need to refer to this specific filing for any reason. 

`expenses_method` (and the redundant `expenses_method_display`) seems to be about whether the registrant wants to report their expenses according to various IRC definitions of lobbying, or the LDA definition. See https://lda.congress.gov/ld/help/default.htm?turl=Documents%2FAppLD2.htm under "Reporting methods." I think we can ignore it, but I keep it for now.

In [87]:
dropped_columns=["url","filing_type_display", "filing_period", "filing_period_display", "filing_document_url", "filing_document_content_type", "expenses_method", "expenses_method_display","posted_by_name", "termination_date" , "registrant_country", "registrant_ppb_country", "registrant_address_1", "registrant_address_2", "registrant_different_address", "registrant_city", "registrant_state", "registrant_zip"]
lobbying_2023_Q2=df.drop(labels=dropped_columns, axis=1)
lobbying_2023_Q2.iloc[57]

filing_uuid                              e7af16b4-d06b-4230-bc30-05d781846aea
filing_type                                                                Q2
filing_year                                                              2023
income                                                                60000.0
expenses                                                                  NaN
dt_posted                                           2023-06-29T13:33:07-04:00
registrant                  {'id': 401103287, 'url': 'https://lda.senate.g...
client                      {'id': 198399, 'url': 'https://lda.senate.gov/...
lobbying_activities         [{'general_issue_code': 'PHA', 'general_issue_...
conviction_disclosures                                                     []
foreign_entities                                                           []
affiliated_organizations                                                   []
Name: 57, dtype: object

This looks a lot less overwhelming to me!

Now, what's going on with the `registrant`, `client`, `lobbying_activities`, `conviction_disclosures`, `foreign_entities`, and `affiliated_organizations` columns?

#### Dealing with dictionaries in the DataFrames

There are list and dictionary literals in the `registrant`, `client`, `lobbying_activities`, `conviction_disclosures`, `foreign_entities`, and `affiliated_organizations` columns columns. We would like to extract out the contents of these to make the data easier to look at.

In [78]:
ast.literal_eval(df.iloc[57].registrant)

{'id': 401103287,
 'url': 'https://lda.senate.gov/api/v1/registrants/401103287/',
 'house_registrant_id': 401103287,
 'name': 'MCDERMOTT+CONSULTING LLC',
 'description': 'Consulting Firm',
 'address_1': '500 North Capitol Street, NW',
 'address_2': None,
 'address_3': None,
 'address_4': None,
 'city': 'Washington',
 'state': 'DC',
 'state_display': 'District of Columbia',
 'zip': '20001',
 'country': 'US',
 'country_display': 'United States of America',
 'ppb_country': 'US',
 'ppb_country_display': 'United States of America',
 'contact_name': 'ERIC ZIMMERMAN',
 'contact_telephone': '+1 202-204-1457',
 'dt_updated': '2024-01-02T12:57:52.578475-05:00'}

From this, we only really want the name and id number. What about `client`?

In [28]:
ast.literal_eval(df.iloc[57].client)

{'id': 198399,
 'url': 'https://lda.senate.gov/api/v1/clients/198399/',
 'client_id': 24,
 'name': 'ALLIANCE FOR RURAL HOSPITAL ACCESS',
 'general_description': 'Coalition of Rural Referral Centers and Sole Community Hospitals',
 'client_government_entity': False,
 'client_self_select': None,
 'state': 'DC',
 'state_display': 'District of Columbia',
 'country': 'US',
 'country_display': 'United States of America',
 'ppb_state': 'DC',
 'ppb_state_display': 'District of Columbia',
 'ppb_country': 'US',
 'ppb_country_display': 'United States of America',
 'effective_date': '2014-04-01'}

Same here: we only want the name and id number.

The `lobbying_activities` column contains a string literal representing a list of dictionaries. We can see that these dictionaries contain a lot of information on the lobbying activity: names of lobbyists, descriptions of the lobbying, and most importantly, the codes describing the lobbying sector.

In [29]:
ast.literal_eval(df.iloc[57].lobbying_activities)

[{'general_issue_code': 'PHA',
  'general_issue_code_display': 'Pharmacy',
  'description': 'Amendments to Section 340B of the Public Health Service Act regarding eligibility for rural referral centers and sole community hospitals in the federal discount drug purchasing program.',
  'foreign_entity_issues': '',
  'lobbyists': [{'lobbyist': {'id': 60423,
     'prefix': None,
     'prefix_display': None,
     'first_name': 'ERIC',
     'nickname': None,
     'middle_name': None,
     'last_name': 'ZIMMERMAN',
     'suffix': None,
     'suffix_display': None},
    'covered_position': None,
    'new': False},
   {'lobbyist': {'id': 60428,
     'prefix': None,
     'prefix_display': None,
     'first_name': 'ERICA',
     'nickname': None,
     'middle_name': None,
     'last_name': 'STOCKER',
     'suffix': None,
     'suffix_display': None},
    'covered_position': '2000-2009: Staff Assistant, Legislative Correspondent, Legislative Assistant, Deputy Chief of Staff, Rep. Jim Saxton',
    'n

Let's extract those codes.

In [37]:
k=len(ast.literal_eval(df.iloc[57].lobbying_activities))
issue_codes= [ast.literal_eval(df.iloc[57].lobbying_activities)[i]['general_issue_code'] for i in range(k)]
issue_codes

['PHA', 'MMM']

Let's also grab the government entities lobbied. I'll drop duplicates by making it a set.

In [134]:
k=len(ast.literal_eval(df.iloc[57].lobbying_activities))
entities= [ast.literal_eval(df.iloc[57].lobbying_activities)[i]['government_entities'] for i in range(k)]
ents=[]
for j in range(len(entities)):
    for k in range(len(entities[j])):
        ents.append(entities[j][k]['name'])
ents=set(ents)

Finally, we take a description of the lobbying activities.

In [100]:
k=len(ast.literal_eval(df.iloc[57].lobbying_activities))
description = [ast.literal_eval(df.iloc[57].lobbying_activities)[i]['description'] for i in range(k)]

Now, we should investigate the often empty `conviction_disclosures`, `foreign_entities`, and `affiliated_organizations`  columns. Entries still look to be lists of dictionaries, like for `lobbying_activities`.

In [17]:
ast.literal_eval(df.loc[df.conviction_disclosures!='[]'].iloc[0]['conviction_disclosures'])

[{'lobbyist': {'id': 29268,
   'prefix': None,
   'prefix_display': None,
   'first_name': 'C',
   'nickname': None,
   'middle_name': 'JAMES',
   'last_name': 'PATTI',
   'suffix': None,
   'suffix_display': None},
  'date': '2002-05-07',
  'description': 'Violation of one count of 18 USC 1341; sentenced to two years of probation'}]

From this, I don't think there is anything I really want - maybe a `True` or `False` depending on whether or not there are any such disclosures. It's extremely unlikely we'll go into tracking individual lobbyists, even if they have convictions.

As for foreign entities, we can record the name and country.

In [79]:
for_ent=ast.literal_eval(df.loc[df.foreign_entities!='[]'].iloc[0]['foreign_entities'])
for_ent

[{'name': 'SK HOLDINGS',
  'contribution': '0.00',
  'ownership_percentage': '100.00',
  'address': '26, Jong-ro, Jongno-gu',
  'city': 'Seoul',
  'state': None,
  'state_display': None,
  'country': 'KR',
  'country_display': 'South Korea',
  'ppb_city': 'Seoul',
  'ppb_state': None,
  'ppb_state_display': None,
  'ppb_country': 'KR',
  'ppb_country_display': 'South Korea'}]

In [90]:
k=len(for_ent)
foreign_entities=[(for_ent[i]['name'],for_ent[i]['country_display']) for i in range(k)]
foreign_entities

[('SK HOLDINGS', 'South Korea')]

In [76]:
aff_org=ast.literal_eval(df.loc[df.affiliated_organizations!='[]'].iloc[0]['affiliated_organizations'])
aff_org

[{'name': 'CAPITOL INTEGRATION',
  'url': None,
  'address_1': '7904 Heritage Grand Place',
  'address_2': None,
  'address_3': None,
  'address_4': None,
  'city': 'Bradenton',
  'state': 'FL',
  'state_display': 'Florida',
  'zip': '34212',
  'country': 'US',
  'country_display': 'United States of America',
  'ppb_city': None,
  'ppb_state': 'FL',
  'ppb_state_display': 'Florida',
  'ppb_country': 'US',
  'ppb_country_display': 'United States of America'}]

In [92]:
k=len(aff_org)
aff_orgs=[aff_org[i]['name'] for i in range(k)]
aff_orgs

['CAPITOL INTEGRATION']

OK, we're ready to clean up some columns. First, a trial run on 2022_Q2.

In [112]:
dropped_columns=["url","filing_type_display", "filing_period", "filing_period_display", "filing_document_url", "filing_document_content_type", "expenses_method_display","posted_by_name", "termination_date","registrant_country", "registrant_ppb_country", "registrant_address_1", "registrant_address_2", "registrant_different_address", "registrant_city", "registrant_state", "registrant_zip"]
df=pd.read_csv("../../lobbying-local-data/LDA_data/Filings_2023/filings_2023_Q2.csv")
reg=[] #these lists will store data for our new columns
affiliated_orgs=[]
cl=[]
foreign_entities=[]
issue_codes=[]
entities_lobbied=[]
lobbying_description=[]
reg_ids=[]
cl_ids=[]
for j in range(len(df)):
    reg.append(ast.literal_eval(df.iloc[j]["registrant"])["name"])
    aff_org=ast.literal_eval(df.iloc[j]['affiliated_organizations']) #grabbing affiliated orgs
    k=len(aff_org)
    aff_orgs_j=[aff_org[i]['name'] for i in range(k)]
    affiliated_orgs.append(aff_orgs_j)
    cl.append(ast.literal_eval(df.iloc[j]["client"])["name"])
    for_ent=ast.literal_eval(df.iloc[j]['foreign_entities']) #grabbing foreign entities
    k=len(for_ent)
    for_ent_j=[(for_ent[i]['name'],for_ent[i]['country_display']) for i in range(k)]
    foreign_entities.append(for_ent_j)
    lobb_act=ast.literal_eval(df.iloc[j]['lobbying_activities']) #extracting out lobbying issue codes from the lobbying_activities column
    k=len(lobb_act) 
    issue_codes_j= [lobb_act[i]['general_issue_code'] for i in range(k)]
    issue_codes.append(issue_codes_j)
    entities=[lobb_act[i]['government_entities'] for i in range(k)] #entities lobbied
    ents=[]
    for l in range(len(entities)):
        for m in range(len(entities[l])):
            ents.append(entities[l][m]['name'])
    ents=list(set(ents))
    ents.sort()
    entities_lobbied.append(ents)
    description_j=[lobb_act[i]['description'] for i in range(k)]
    lobbying_description.append(description_j)
    reg_ids.append(ast.literal_eval(df.iloc[j]["registrant"])["id"])
    cl_ids.append(ast.literal_eval(df.iloc[j]["client"])["id"])

df["registrant"]=reg      #making new columns
df["affiliated_organizations"]=affiliated_orgs
df["client"]=cl
df["foreign_entities"]=foreign_entities
df["issue_codes"]=issue_codes
df["entities_lobbied"]=entities_lobbied
df["lobbying_description"]=lobbying_description
df["reg_id"]=reg_ids
df["cl_id"]=cl_ids
df["JACK_conviction"]=False #JACK_conviction just keeps track of if there is a non-empty entry in conviction_disclosures
df.loc[df.conviction_disclosures!='[]', 'JACK_conviction']=True

df=df.drop(labels=dropped_columns, axis=1) #get rid of the useless columns
df=df.drop(labels=['lobbying_activities','conviction_disclosures'], axis=1) #also drop lobbying_activities and conviction_disclosures as we have gotten what we need from them
df=df.fillna(0) #fills all missing values with 0 - NaNs were only in income, expenses, and expenses_method

Let me write a small function that computes, given a dataframe of filings `df` and a row `i`, the range of dates the appropriate quarter covers. I'll encode this range as a dictionary storing a start and end date.

In [45]:
def filing_period(df,i):
    yr=df.iloc[i]['filing_year']
    quarter=rel_2_stand[df.iloc[i]['filing_type']][0]
    if quarter=='1':
        d= {'start' : datetime.date(year=yr, month=1, day=1), 'end' : datetime.date(year=yr, month=3, day=31)}
    if quarter=='2':
        d= {'start' : datetime.date(year=yr, month=4, day=1), 'end' : datetime.date(year=yr, month=6, day=30)}
    if quarter=='3':
        d= {'start' : datetime.date(year=yr, month=7, day=1), 'end' : datetime.date(year=yr, month=9, day=30)}
    if quarter=='4':
        d= {'start' : datetime.date(year=yr, month=10, day=1), 'end' : datetime.date(year=yr, month=12, day=31)}
    return d

For our purposes this is overkill - all of our dataframes up to now have the same year and quarter. Still, let's use this to replace the filing year column with a column that tracks the filing period in this format.

In [103]:
df['filing_period']=[filing_period(df,i) for i in range(len(df))] #could do 0 instead of i since all filings in this dataframe have the same period

In [104]:
df=df.drop(labels=['filing_year'], axis=1)

We now reorder the columns so that they are easier to read at a glance.

In [105]:
cols=list(df.columns.values)
print(list(enumerate(cols)))

[(0, 'filing_uuid'), (1, 'filing_type'), (2, 'income'), (3, 'expenses'), (4, 'expenses_method'), (5, 'dt_posted'), (6, 'registrant'), (7, 'client'), (8, 'foreign_entities'), (9, 'affiliated_organizations'), (10, 'issue_codes'), (11, 'entities_lobbied'), (12, 'lobbying_description'), (13, 'reg_id'), (14, 'cl_id'), (15, 'JACK_conviction'), (16, 'filing_period')]


In [106]:
cols=cols[6:8]+cols[2:4]+cols[10:13]+cols[8:10]+cols[15:16]+cols[0:2]+cols[4:6]+cols[13:15]

In [107]:
cols

['registrant',
 'client',
 'income',
 'expenses',
 'issue_codes',
 'entities_lobbied',
 'lobbying_description',
 'foreign_entities',
 'affiliated_organizations',
 'JACK_conviction',
 'filing_uuid',
 'filing_type',
 'expenses_method',
 'dt_posted',
 'reg_id',
 'cl_id']

In [108]:
df=df[cols]

OK, let's assemble this all into a function that cleans a dataframe.

In [7]:
def filing_period(df,i):
    yr=df.iloc[i]['filing_year']
    quarter=rel_2_stand[df.iloc[i]['filing_type']][0]
    if quarter=='1':
        d= {'start' : datetime.date(year=yr, month=1, day=1), 'end' : datetime.date(year=yr, month=3, day=31)}
    if quarter=='2':
        d= {'start' : datetime.date(year=yr, month=4, day=1), 'end' : datetime.date(year=yr, month=6, day=30)}
    if quarter=='3':
        d= {'start' : datetime.date(year=yr, month=7, day=1), 'end' : datetime.date(year=yr, month=9, day=30)}
    if quarter=='4':
        d= {'start' : datetime.date(year=yr, month=10, day=1), 'end' : datetime.date(year=yr, month=12, day=31)}
    return d

def clean_cols(df):
    dropped_columns=["url","filing_type_display", "filing_period", "filing_period_display", "filing_document_url", "filing_document_content_type", "expenses_method_display","posted_by_name", "termination_date","registrant_country", "registrant_ppb_country", "registrant_address_1", "registrant_address_2", "registrant_different_address", "registrant_city", "registrant_state", "registrant_zip"]
    reg=[] #these lists will store data for our new columns
    affiliated_orgs=[]
    cl=[]
    foreign_entities=[]
    issue_codes=[]
    entities_lobbied=[]
    lobbying_description=[]
    reg_ids=[]
    cl_ids=[]
    dts=[]
    for j in range(len(df)):
        reg.append(ast.literal_eval(df.iloc[j]["registrant"])["name"])
        aff_org=ast.literal_eval(df.iloc[j]['affiliated_organizations']) #grabbing affiliated orgs
        k=len(aff_org)
        aff_orgs_j=[aff_org[i]['name'] for i in range(k)]
        affiliated_orgs.append(aff_orgs_j)
        cl.append(ast.literal_eval(df.iloc[j]["client"])["name"])
        for_ent=ast.literal_eval(df.iloc[j]['foreign_entities']) #grabbing foreign entities
        k=len(for_ent)
        for_ent_j=[(for_ent[i]['name'],for_ent[i]['country_display']) for i in range(k)]
        foreign_entities.append(for_ent_j)
        lobb_act=ast.literal_eval(df.iloc[j]['lobbying_activities']) #extracting out lobbying issue codes from the lobbying_activities column
        k=len(lobb_act) 
        issue_codes_j= [lobb_act[i]['general_issue_code'] for i in range(k)]
        issue_codes.append(issue_codes_j)
        entities=[lobb_act[i]['government_entities'] for i in range(k)] #entities lobbied
        ents=[]
        for l in range(len(entities)):
            for m in range(len(entities[l])):
                ents.append(entities[l][m]['name'])
        ents=list(set(ents))
        ents.sort()
        entities_lobbied.append(ents)
        description_j=[lobb_act[i]['description'] for i in range(k)]
        lobbying_description.append(description_j)
        reg_ids.append(ast.literal_eval(df.iloc[j]["registrant"])["id"])
        cl_ids.append(ast.literal_eval(df.iloc[j]["client"])["id"])
        dts.append(df.iloc[j]["dt_posted"][:-6]) #cleaning the dt_posted column

    df["registrant"]=reg      #making new columns
    df["affiliated_organizations"]=affiliated_orgs
    df["client"]=cl
    df["foreign_entities"]=foreign_entities
    df["issue_codes"]=issue_codes
    df["entities_lobbied"]=entities_lobbied
    df["lobbying_description"]=lobbying_description
    df["reg_id"]=reg_ids
    df["cl_id"]=cl_ids
    df["JACK_conviction"]=False #JACK_conviction just keeps track of if there is a non-empty entry in conviction_disclosures
    df.loc[df.conviction_disclosures!='[]', 'JACK_conviction']=True
    df["dt_posted"]=dts
    df['dt_posted']=pd.to_datetime(df['dt_posted'], format='%Y-%m-%dT%H:%M:%S') #turning the dt_posted column into datetimes

    df=df.drop(labels=dropped_columns, axis=1) #get rid of the useless columns
    df=df.drop(labels=['lobbying_activities','conviction_disclosures'], axis=1) #also drop lobbying_activities and conviction_disclosures as we have gotten what we need from them
    df=df.fillna(0) #fills all missing values with 0 - NaNs were only in income, expenses, and expenses_method
    df['filing_period']=[filing_period(df,i) for i in range(len(df))] #could do 0 instead of i since all filings in this dataframe have the same period
    df=df.drop(labels=['filing_year'], axis=1)
    cols=list(df.columns.values)
    cols=cols[6:8]+cols[2:4]+cols[10:13]+cols[8:10]+cols[15:16]+cols[0:2]+cols[4:6]+cols[13:15]
    df=df[cols]
    return df
    

## (3) Row hygiene

There are duplicate entries in the filings. For instance, look at our dataframe again.

In [139]:
df=clean_cols(pd.read_csv("../../lobbying-local-data/LDA_data/Filings_2023/filings_2023_Q2.csv"))

In [138]:
type(df.iloc[2]['dt_posted'])

pandas._libs.tslibs.timestamps.Timestamp

In [127]:
type(df.loc[2]['dt_posted'])

pandas._libs.tslibs.timestamps.Timestamp

In [116]:
ids=set() #ids is a set to avoid double-counting
duplicated_ids=pd.DataFrame()
for i in range(len(df)):
    ids.add((df.iloc[i]['reg_id'],df.iloc[i]['cl_id']))  #we add data as a tuple, since sets of lists are not allowed in python
ids_list = list(ids) #turning ids into a list so we can iterate through the entries (sets are unordered)
for j in range(len(ids_list)):
    if len(df.loc[(df.reg_id==ids_list[j][0]) & (df.cl_id==ids_list[j][1]) ])>1:
        duplicated_ids=pd.concat([duplicated_ids, df.loc[(df.reg_id==ids_list[j][0]) & (df.cl_id==ids_list[j][1])]])
duplicated_ids

,registrant,client,income,expenses,issue_codes,entities_lobbied,lobbying_description,foreign_entities,affiliated_organizations,JACK_conviction,filing_uuid,filing_type,expenses_method,dt_posted,reg_id,cl_id
4028,O'NEILL AND ASSOCIATES,"EV TRANSPORTATION SERVICES, INC.",15000.0,0.0,[POS],"[HOUSE OF REPRESENTATIVES, SENATE]",[Electric vehicle/delivery transportation-rela...,[],[],False,c483d856-f404-42ec-8802-35fef8d39638,Q2,0,2023-07-17T14:43:49-04:00,73134,54488
4031,O'NEILL AND ASSOCIATES,"EV TRANSPORTATION SERVICES, INC.",15000.0,0.0,[POS],"[HOUSE OF REPRESENTATIVES, SENATE, U.S. Postal...",[Electric vehicle/delivery transportation-rela...,[],[],False,558f9d48-bd99-49b0-8d34-eee6b8582d3d,Q2,0,2023-07-17T14:44:59-04:00,73134,54488
953,"SHORELIGHT, LLC (FKA SHORELIGHT EDUCATION, LLC)","SHORELIGHT, LLC (FKA SHORELIGHT EDUCATION, LLC)",0.0,48700.0,"[EDU, IMM]","[Commerce, Dept of (DOC), HOUSE OF REPRESENTAT...",[Immigration pertaining to international stude...,[],[],False,ee7c7e06-e929-4b81-a5e8-dd680d742241,Q2,A,2023-07-08T19:07:34-04:00,401104424,201766
1080,"SHORELIGHT, LLC (FKA SHORELIGHT EDUCATION, LLC)","SHORELIGHT, LLC (FKA SHORELIGHT EDUCATION, LLC)",0.0,48700.0,"[EDU, IMM]","[Commerce, Dept of (DOC), HOUSE OF REPRESENTAT...",[Immigration pertaining to international stude...,[],[],False,4bab4e80-93b4-4749-8b60-a707aa0b0222,Q2,A,2023-07-10T10:21:17-04:00,401104424,201766
1121,"LANTHEUS HOLDINGS, INC.","LANTHEUS HOLDINGS, INC.",0.0,110000.0,[MMM],"[HOUSE OF REPRESENTATIVES, SENATE]",[Medicare HOPPS separate payment for high-prec...,[],[],False,458693e5-5a07-4757-bad8-d2510a4a1960,Q2,A,2023-07-10T12:52:21-04:00,401105198,203388
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26,KERRY O'BRIEN,NATIONAL DOMESTIC WORKERS ALLIANCE,10000.0,0.0,[LBR],"[HOUSE OF REPRESENTATIVES, Office of Science &...",[Rights and responsibilities of domestic worke...,[],[],False,4e56b143-bfe3-4e4b-a046-43bbd9d4aeaa,Q2,0,2023-06-25T13:21:49-04:00,401105600,55027
4165,MCCARTHY STRATEGIC SOLUTIONS,"INOTIV, INC.",0.0,0.0,"[MED, ANI]","[HOUSE OF REPRESENTATIVES, SENATE]","[Animal research models to biopharmaceuticals,...",[],[],False,44aa6361-a9b5-4470-940c-00fcc007ba7f,Q2,0,2023-07-17T15:29:46-04:00,300936,213558
4169,MCCARTHY STRATEGIC SOLUTIONS,"INOTIV, INC.",0.0,0.0,"[MED, ANI]","[HOUSE OF REPRESENTATIVES, SENATE]","[Animal research models to biopharmaceuticals,...",[],[],False,acd750d5-b246-4195-9172-d59157f89797,Q2,0,2023-07-17T15:31:06-04:00,300936,213558
6997,"PUBLIC STRATEGIES WASHINGTON, INC.",NOVO NORDISK INC.,30000.0,0.0,[MMM],"[HOUSE OF REPRESENTATIVES, SENATE]",[Issues related to the coverage of obesity the...,[],[],False,55c006b4-5f6d-44e6-ade3-09b492bc80ca,Q2,0,2023-07-19T11:03:24-04:00,32486,53343


In [117]:
duped_ids=set()
for i in range(len(duplicated_ids)):
    duped_ids.add((duplicated_ids.iloc[i]['reg_id'],duplicated_ids.iloc[i]['cl_id']))
duped_list = list(duped_ids)
len(duped_list)

45

Here, 45 filings were actually filed twice. These are probably just due to some human error. But, in theory, there may be slight variations between these filings - we can stare at `duplicated_ids` to see this.

OK, moving forward we will just use the `drop_duplicates` function to remove dupes. We will keep the last, since the dataframes are all ordered by `dt_posted` naturally.

## (4) Updating quarterly filings according to amendments

Recall that some filings (with codes involving A or @) are amendments to previous filings. Let's take a look at one of these:

In [13]:
df_A = clean_cols(pd.read_csv("../../lobbying-local-data/LDA_data/Filings_2023/filings_2023_2A.csv"))
dropped_A=df_A.drop_duplicates(subset=['reg_id','cl_id'], keep='last')
print(df_A.info())
print()
print(dropped_A.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 608 entries, 0 to 607
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   registrant                608 non-null    object 
 1   client                    608 non-null    object 
 2   income                    608 non-null    float64
 3   expenses                  608 non-null    float64
 4   issue_codes               608 non-null    object 
 5   entities_lobbied          608 non-null    object 
 6   lobbying_description      608 non-null    object 
 7   foreign_entities          608 non-null    object 
 8   affiliated_organizations  608 non-null    object 
 9   JACK_conviction           608 non-null    bool   
 10  filing_uuid               608 non-null    object 
 11  filing_type               608 non-null    object 
 12  expenses_method           608 non-null    object 
 13  dt_posted                 608 non-null    object 
 14  reg_id    

There are 608 amendments filed in Q2 of 2023, of these, there are 541 non-duplicates.

In [46]:
df=clean_cols(pd.read_csv("../../lobbying-local-data/LDA_data/Filings_2023/filings_2023_Q2.csv"))
dropped=df.drop_duplicates(subset=['reg_id','cl_id'], keep='last')
print(df.info())
print()
print(dropped.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18362 entries, 0 to 18361
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   registrant                18362 non-null  object 
 1   client                    18362 non-null  object 
 2   income                    18362 non-null  float64
 3   expenses                  18362 non-null  float64
 4   issue_codes               18362 non-null  object 
 5   entities_lobbied          18362 non-null  object 
 6   lobbying_description      18362 non-null  object 
 7   foreign_entities          18362 non-null  object 
 8   affiliated_organizations  18362 non-null  object 
 9   JACK_conviction           18362 non-null  bool   
 10  filing_uuid               18362 non-null  object 
 11  filing_type               18362 non-null  object 
 12  expenses_method           18362 non-null  object 
 13  dt_posted                 18362 non-null  object 
 14  reg_id

There are 18362 Q2 filings for 2023, of which 18317 are non-duplicates. How many of these correspond to entries in "lobbying_2023_2A"? We can check.

In [15]:
df_toamend=pd.DataFrame()
df_fromamend=pd.DataFrame()
for i in range(len(dropped_A)):
    reg_id_to_find=dropped_A.iloc[i]['reg_id']
    cl_id_to_find=dropped_A.iloc[i]['cl_id']
    dropped_A_i=pd.DataFrame(dropped_A.iloc[[i]])
    dropped_i=pd.DataFrame(dropped.loc[(dropped.reg_id==reg_id_to_find) &  (dropped.cl_id==cl_id_to_find)])
    df_toamend=pd.concat([df_toamend,dropped_i])
    if len(dropped_i)>0:
        df_fromamend=pd.concat([df_fromamend, dropped_A_i])

Here we make two dataframes: the first, "df_toamend", lists all the original filings in Q2 2023 that match the registrant and client ids for an amendment filed in Q2 2023; the second, "df_fromamend," keeps track of all of the amendments matching those original filings.

Just a sanity check: these two should have the same number of filings in them.

In [16]:
print(len(df_toamend))
print()
print(len(df_fromamend))

432

432


Let's just take a peek at some of the amendments and the filings they are modifying, just to make sure everything is lining up correctly.

In [17]:
k=np.random.randint(0,423)
df_toamend.iloc[range(k,k+10)]

,registrant,client,income,expenses,issue_codes,entities_lobbied,lobbying_description,foreign_entities,affiliated_organizations,JACK_conviction,filing_uuid,filing_type,expenses_method,dt_posted,reg_id,cl_id
13009,"ARENA STRATEGIES, LLC",INDEPENDENCE BLUE CROSS,18750.0,0.0,"[MMM, INS, HCR]","[HOUSE OF REPRESENTATIVES, SENATE]","[Medicare/Medicaid, Insurance, Health Care]",[],[],False,3e5bb090-91c1-49fd-b9e4-ae76a3c86a2b,Q2,0,2023-07-20T13:50:31-04:00,401103555,199522
2951,EMERGENCY DEPARTMENT PRACTICE MANAGEMENT ASSOC...,EMERGENCY DEPARTMENT PRACTICE MANAGEMENT ASSOC...,0.0,6000.0,[HCR],[Centers For Medicare and Medicaid Services (C...,[Implementation of the No Surprises Act Telehe...,[],[],False,a59e47b5-1495-4d6f-a36f-cae1aed6470e,Q2,A,2023-07-14T17:50:53-04:00,400316484,189221
6166,HARDWOOD FEDERATION,HARDWOOD FEDERATION,0.0,148000.0,"[AGR, TRD, TRU, TAX]","[Agriculture, Dept of (USDA), Federal Maritime...",[Farm Bill Reauthorization - wood products-rel...,[],[],False,f5569ccb-7c99-436e-a0aa-994cf79b83b0,Q2,A,2023-07-18T20:47:55-04:00,300515,180393
15976,FILECOIN FOUNDATION,FILECOIN FOUNDATION,0.0,70000.0,"[SCI, FIN, CPI, CDT, BAN]","[HOUSE OF REPRESENTATIVES, SENATE]",[General Discussions related to digital assets...,[],[],False,5e7acd0c-30f5-40e5-ae7d-1cdd16f5a062,Q2,A,2023-07-20T19:58:14-04:00,401107131,53653
1266,MG HOUSING STRATEGIES LLC,CREA,20000.0,0.0,[HOU],"[HOUSE OF REPRESENTATIVES, SENATE, White House...",[Low-Income Housing Tax Credit program],[],[],False,19ab4205-bd27-4252-a5ae-68eda91b50df,Q2,0,2023-07-11T09:59:29-04:00,401105996,209301
1267,MG HOUSING STRATEGIES LLC,INLAND GROUP,10000.0,0.0,[HOU],"[HOUSE OF REPRESENTATIVES, SENATE]",[Low-Income Housing Tax Credit program],[],[],False,7b1d67e9-9bf4-460e-8061-34cfa0dffa95,Q2,0,2023-07-11T10:00:09-04:00,401105996,209218
1268,MG HOUSING STRATEGIES LLC,THE PACIFIC COMPANIES,7500.0,0.0,[HOU],"[HOUSE OF REPRESENTATIVES, SENATE]",[Low-Income Housing Tax Credit program],[],[],False,34edea14-77d8-4d3e-8f59-e5ffb193667a,Q2,0,2023-07-11T10:00:38-04:00,401105996,209224
1269,MG HOUSING STRATEGIES LLC,WENDOVER HOUSING PARTNERS,10000.0,0.0,[HOU],"[HOUSE OF REPRESENTATIVES, SENATE]",[Low-Income Housing Tax Credit program],[],[],False,608a9755-bc7b-4e17-b340-7a985d5c5507,Q2,0,2023-07-11T10:05:31-04:00,401105996,209221
1270,MG HOUSING STRATEGIES LLC,THE NRP GROUP,10000.0,0.0,[HOU],"[HOUSE OF REPRESENTATIVES, SENATE]",[Low-Income Housing Tax Credit program],[],[],False,20024709-256a-4e41-b09e-8adca800086e,Q2,0,2023-07-11T10:06:00-04:00,401105996,209217
1271,MG HOUSING STRATEGIES LLC,CHURCHILL STATESIDE,5000.0,0.0,[HOU],"[HOUSE OF REPRESENTATIVES, SENATE]",[Low-Income Housing Tax Credit program],[],[],False,fa1f318b-14b6-40e6-b8c7-15d7c0a84a22,Q2,0,2023-07-11T10:06:45-04:00,401105996,209219


In [ ]:
df_fromamend.iloc[range(k,k+10)]

Looks OK.

Note that of the 541 amendments, 432 modify filings in Q2 2023. The remaining 109 amendments we store as a new dataframe.

In [19]:
rem=pd.concat([dropped_A,df_fromamend]).drop_duplicates(subset=['reg_id','cl_id'],keep=False)

In [20]:
len(rem)

109

We now hunt for where the corresponding files to amend are.

In [21]:
df_Y=clean_cols(pd.read_csv("../../lobbying-local-data/LDA_data/Filings_2023/filings_2023_Q2Y.csv"))
dropped_Y=df_Y.drop_duplicates(subset=['reg_id','cl_id'], keep='last')

In [22]:
rem_to_amend=pd.DataFrame()
rem_from_amend=pd.DataFrame()
for i in range(len(rem)):
    reg_id_to_find=rem.iloc[i]['reg_id']
    cl_id_to_find=rem.iloc[i]['cl_id']
    rem_i=pd.DataFrame(rem.iloc[[i]])
    dropped_Y_i=pd.DataFrame(dropped_Y.loc[(dropped_Y.reg_id==reg_id_to_find) &  (dropped_Y.cl_id==cl_id_to_find)])
    rem_to_amend=pd.concat([rem_to_amend,dropped_Y_i])
    if len(dropped_Y_i)>0:
        rem_from_amend=pd.concat([rem_from_amend, rem_i])

We take a look:

In [23]:
pd.concat([rem_from_amend,rem_to_amend]).sort_values(['reg_id','cl_id'])

,registrant,client,income,expenses,issue_codes,entities_lobbied,lobbying_description,foreign_entities,affiliated_organizations,JACK_conviction,filing_uuid,filing_type,expenses_method,dt_posted,reg_id,cl_id
521,APCO WORLDWIDE LLC,"LIMINEX, INC. DBA GOGUARDIAN",30000.0,0.0,"[EDU, TEC]",[],[General issues related to ed tech and school ...,[],[],False,8dddddfb-22cd-48db-b1b7-f0f60c0450b4,2A,0,2023-11-13T14:09:30-05:00,4117,55782
981,APCO WORLDWIDE LLC,"LIMINEX, INC. DBA GOGUARDIAN",0.0,0.0,[],[],[],[],[],False,d1a5e1dd-1012-472d-a132-2cdf9e271373,Q2Y,0,2023-07-19T13:28:18-04:00,4117,55782
435,BRACEWELL LLP,DENKA PERFORMANCE ELASTOMER LLC,65000.0,0.0,"[BUD, CHM]","[Environmental Protection Agency (EPA), HOUSE ...",[Appropriations issues related to Appropriatio...,[],[],False,2102afe4-4ad8-4dd3-abfa-e5e3ab8b8fdc,2A,0,2023-10-18T16:16:45-04:00,6837,109447
1851,BRACEWELL LLP,DENKA PERFORMANCE ELASTOMER LLC,0.0,0.0,[],[],[],[],[],False,32622d24-168c-435f-9728-88722e2e69e1,Q2Y,0,2023-07-20T18:54:56-04:00,6837,109447
366,NATURAL RESOURCES DEFENSE COUNCIL,NATURAL RESOURCES DEFENSE COUNCIL,0.0,276960.0,"[TAX, GOV, ECN]","[HOUSE OF REPRESENTATIVES, SENATE]",[Debt Limit - H.R. 415 - IRA defense. IRA defe...,[],[],False,43d5d9b0-f048-4180-9f2a-3515793a542e,2A,B,2023-09-11T09:22:48-04:00,28867,134600
1754,NATURAL RESOURCES DEFENSE COUNCIL,NATURAL RESOURCES DEFENSE COUNCIL,0.0,276960.0,[],[],[],[],[],False,9f01dbb4-c880-4ab7-944b-63792aea32fd,Q2Y,B,2023-07-20T16:04:08-04:00,28867,134600
285,CARDINAL INFRASTRUCTURE LLC,THE ROUTING COMPANY,0.0,0.0,"[BUD, TRA]","[Federal Transit Administration (FTA), HOUSE O...","[THUD programmatic appropriations request, Aut...",[],[],False,3c04e249-1501-4b5c-b117-8e5da723b24f,2A,0,2023-08-04T11:57:38-04:00,299913,54172
2060,CARDINAL INFRASTRUCTURE LLC,THE ROUTING COMPANY,0.0,0.0,[],[],[],[],[],False,8c04ef28-f66c-4034-bb8a-fc2d0116fa3c,Q2Y,0,2023-08-04T11:55:21-04:00,299913,54172
362,"JILL WARREN, P.C.","ICON AIRCRAFT, INC.",0.0,0.0,[AVI],[],[FAA Aircraft Certification],[(PUDONG SCIENCE AND TECHNOLOGY INVESTMENT CO....,[],False,ea25a1fd-dbd8-4362-8f53-9eed3922574e,2A,0,2023-09-05T23:20:01-04:00,323012,56714
2342,"JILL WARREN, P.C.","ICON AIRCRAFT, INC.",0.0,0.0,[],[],[],[],[],False,6dc0f789-9864-47c0-bbff-1bcc376a3f31,Q2Y,0,2023-09-05T23:02:23-04:00,323012,56714


This takes care of 8 more. We keep the remaining 101 as a dataframe.

In [25]:
rem_Y=pd.concat([rem,rem_from_amend]).drop_duplicates(subset=['reg_id','cl_id'],keep=False)

In [27]:
df_T=clean_cols(pd.read_csv("../../lobbying-local-data/LDA_data/Filings_2023/filings_2023_2T.csv"))
dropped_T=df_T.drop_duplicates(subset=['reg_id','cl_id'],keep='last')

In [28]:
rem_Y_to_amend=pd.DataFrame()
rem_Y_from_amend=pd.DataFrame()
for i in range(len(rem_Y)):
    reg_id_to_find=rem_Y.iloc[i]['reg_id']
    cl_id_to_find=rem_Y.iloc[i]['cl_id']
    rem_Y_i=pd.DataFrame(rem_Y.iloc[[i]])
    dropped_T_i=pd.DataFrame(dropped_T.loc[(dropped_T.reg_id==reg_id_to_find) &  (dropped_T.cl_id==cl_id_to_find)])
    rem_Y_to_amend=pd.concat([rem_Y_to_amend,dropped_T_i])
    if len(dropped_T_i)>0:
        rem_Y_from_amend=pd.concat([rem_Y_from_amend, rem_Y_i])
pd.concat([rem_Y_from_amend,rem_Y_to_amend]).sort_values(['reg_id','cl_id'])

,registrant,client,income,expenses,issue_codes,entities_lobbied,lobbying_description,foreign_entities,affiliated_organizations,JACK_conviction,filing_uuid,filing_type,expenses_method,dt_posted,reg_id,cl_id
264,MARIJUANA POLICY PROJECT,MARIJUANA POLICY PROJECT,0.0,0.0,"[AGR, BAN]",[HOUSE OF REPRESENTATIVES],"[H.R.2 - Agriculture Improvement Act of 2018, ...",[],[],False,8c741c5d-cffe-4a43-8479-bab8a8483ca4,2A,A,2023-07-28T14:27:37-04:00,74330,167449
448,MARIJUANA POLICY PROJECT,MARIJUANA POLICY PROJECT,0.0,0.0,"[AGR, BAN]",[HOUSE OF REPRESENTATIVES],[Made recommended improvements to the Agricult...,[],[],False,97b5db56-a2b5-42f8-be72-aba8f7ad9fc6,2T,A,2023-09-14T21:22:43-04:00,74330,167449
214,MCLARTY INBOUND LLC,"AN ACQUISITION, LLC",130000.0,0.0,[GOV],"[State, Dept of (DOS), Treasury, Dept of]",[To provide advocacy support for the potential...,[],[],False,4c2fdb6c-1cf9-4aec-b5a5-0cee8e27fca5,2A,0,2023-07-20T21:41:46-04:00,401103585,55982
464,MCLARTY INBOUND LLC,"AN ACQUISITION, LLC",130000.0,0.0,[GOV],"[State, Dept of (DOS), Treasury, Dept of]",[To provide advocacy support for the potential...,[],[],False,6e2c4002-c394-4dba-bc66-dc191e64783b,2T,0,2023-10-19T19:30:36-04:00,401103585,55982


That's 2 more. 99 to go. Let's look at the previous quarters filings, and see if these amendments are meant to apply to them.

In [29]:
rem_YT=pd.concat([rem_Y,rem_Y_from_amend]).drop_duplicates(subset=['reg_id','cl_id'],keep=False)

In [31]:
df_1=clean_cols(pd.read_csv("../../lobbying-local-data/LDA_data/Filings_2023/filings_2023_Q1.csv"))
dropped_1=df_1.drop_duplicates(subset=['reg_id','cl_id'],keep='last')

In [32]:
rem_YT_to_amend=pd.DataFrame()
rem_YT_from_amend=pd.DataFrame()
for i in range(len(rem_YT)):
    reg_id_to_find=rem_YT.iloc[i]['reg_id']
    cl_id_to_find=rem_YT.iloc[i]['cl_id']
    rem_YT_i=pd.DataFrame(rem_YT.iloc[[i]])
    dropped_1_i=pd.DataFrame(dropped_1.loc[(dropped_1.reg_id==reg_id_to_find) &  (dropped_1.cl_id==cl_id_to_find)])
    rem_YT_to_amend=pd.concat([rem_YT_to_amend,dropped_1_i])
    if len(dropped_1_i)>0:
        rem_YT_from_amend=pd.concat([rem_YT_from_amend, rem_YT_i])
pd.concat([rem_YT_from_amend,rem_YT_to_amend]).sort_values(['reg_id','cl_id'])

,registrant,client,income,expenses,issue_codes,entities_lobbied,lobbying_description,foreign_entities,affiliated_organizations,JACK_conviction,filing_uuid,filing_type,expenses_method,dt_posted,reg_id,cl_id
595,CREDIT UNION NATIONAL ASSOCIATION,CREDIT UNION NATIONAL ASSOCIATION,0.0,1010000.0,"[TAX, SMB, BUD, BAN]","[Consumer Financial Protection Bureau (CFPB), ...","[Credit Union Tax Exemption, 1071 Related Issu...",[],[],False,5b20c9fa-6b7b-4716-899c-f83c5b7273fd,2A,A,2024-01-22T12:40:00-05:00,11322,115381
9756,CREDIT UNION NATIONAL ASSOCIATION,CREDIT UNION NATIONAL ASSOCIATION,0.0,1030678.0,"[TAX, SMB, FOR, BUD, BAN]","[Agriculture, Dept of (USDA), Consumer Financi...","[Credit Union Tax Status, SBA 7a Loan Issues, ...",[],[],False,67b5c440-8621-488d-94ae-c02668f9e295,Q1,A,2023-04-20T10:20:47-04:00,11322,115381
219,EXXON MOBIL CORP,EXXON MOBIL CORP,0.0,1300000.0,"[TRA, TAX, SCI, RRR, FUE, FOR, ENV, ENG, DEF, ...","[Commerce, Dept of (DOC), Council on Environme...","[Discussions related to low emissions fuels, P...",[],[],False,85df947c-0609-4538-83f1-c18b9ae879a1,2A,A,2023-07-21T11:49:56-04:00,14017,119494
15194,EXXON MOBIL CORP,EXXON MOBIL CORP,0.0,1600000.0,"[TRA, TAX, RRR, NAT, FUE, FOR, ENV, ENG, CHM, ...","[Energy, Dept of, Environmental Protection Age...","[Discussions related to pipeline safety, Publi...",[],[],False,352b6c7c-9c25-4434-a066-dd4986381c08,Q1,A,2023-04-20T20:13:34-04:00,14017,119494
229,HOGAN LOVELLS US LLP,GROWTH ENERGY,60000.0,0.0,[TAX],"[HOUSE OF REPRESENTATIVES, SENATE]",[Issues related to the interpretation of the I...,[],[],False,8a2421db-65fb-402b-abd0-ceb0a3d6a03c,2A,0,2023-07-21T16:23:49-04:00,18422,55350
7197,HOGAN LOVELLS US LLP,GROWTH ENERGY,20000.0,0.0,[TAX],"[HOUSE OF REPRESENTATIVES, SENATE, Treasury, D...",[Issues related to the interpretation of the I...,[],[],False,c982957d-fbf4-4e7b-aa52-5e6b6079b2ef,Q1,0,2023-04-19T15:06:42-04:00,18422,55350
474,CLIFF MADISON,NEWHALL LAND AND FARMING COMPANYU,24000.0,0.0,[TRA],"[HOUSE OF REPRESENTATIVES, SENATE]",[fully fund FY 24 transportation appropriation...,[],[],False,86781847-559a-42fb-9f73-56395f9c8083,2A,0,2023-10-22T15:11:34-04:00,23463,130216
17937,CLIFF MADISON,NEWHALL LAND AND FARMING COMPANYU,24000.0,0.0,[TRA],"[HOUSE OF REPRESENTATIVES, SENATE]",[fully fund FY 24 transportation appropriation...,[],[],False,d4d62993-d45d-41a9-a15f-6cca7de22370,Q1,0,2023-07-23T15:52:43-04:00,23463,130216
241,SOCIETY OF CHEMICAL MANUFACTURERS & AFFILIATES,SOCIETY OF CHEMICAL MANUFACTURERS & AFFILIATES,0.0,10000.0,"[TRD, TAR, SMB, MAN, ENV, CHM, FUE, CAW, BUD]","[Commerce, Dept of (DOC), Environmental Protec...","[301 tariffs MTB GSP, Miscellaneous tariff bil...",[],[],False,6a83e87b-8583-45b9-adf6-ad0e839b0a24,2A,A,2023-07-25T13:36:36-04:00,37538,146406
16688,SOCIETY OF CHEMICAL MANUFACTURERS & AFFILIATES,SOCIETY OF CHEMICAL MANUFACTURERS & AFFILIATES,0.0,10000.0,"[TRD, TAR, SMB, MAN, ENV, CHM, FUE, CAW, BUD]","[Commerce, Dept of (DOC), Environmental Protec...","[301 tariffs, Miscellaneous tariff bill, marke...",[],[],False,3dbb33e3-a9ad-49ab-848b-7e7498afc4cd,Q1,A,2023-05-01T12:50:50-04:00,37538,146406


This is extremely unconvincing. More likely, it seems that these 99 filings were meant to be standard Q2 filings, but the "amendment" box on the form accidentally was ticked. In fact, based on this (and some other playing around with the data), I'm going to make a blanket call that amendment filings reference only the quarter in which they are filed.

That's enough exploration.

Let's now formalize what our process looks like for incorporating amendments. First, we will look at the correct place for each amendment type, i.e. for 2A we look at 2, for 2AY we look at 2Y, for 2@ we look at 2T, for 2@Y we look at 2TY. We update 2, 2Y, 2T, 2TY accordingly (we can do this by e.g. concatenating 2 and the matching filings in 2A, then dropping duplicates keeping the first). There will be leftovers, which I call `rem_A`, `rem_AY`, `rem_@`, `rem_@Y`. From these, we make a full collection of remainders `rem_all` by concatenating `rem_A`, `rem_AY`, `rem_@`, `rem_@Y` and dropping duplicates. Finally, we look at the unmatched filings in 2, 2Y, 2T, 2TY, which I call `unmatch`, `unmatch_Y`, `unmatch_T`, `unmatch_TY`, concatenate and drop duplicates to get `unmatch_all`, then search `unmatch_all` for filings in `rem_all` and replace. What is left over I will call `true_rem`, and simply add it to the dataframe.

In [62]:
filing_suffixes=['','Y','T','TY']
amendment_suffixes=['A','AY','@', '@Y']
fil_2_am=dict(zip(filing_suffixes,amendment_suffixes))


relevant_filing_types=["Q1","Q1Y","1T","1TY","1A","1AY","1@","1@Y","Q2","Q2Y","2T","2TY","2A","2AY","2@","2@Y","Q3","Q3Y","3T","3TY","3A","3AY","3@","3@Y","Q4","Q4Y","4T","4TY","4A","4AY","4@","4@Y"]
stand_filing_types=["1","1Y","1T","1TY","1A","1AY","1@","1@Y","2","2Y","2T","2TY","2A","2AY","2@","2@Y","3","3Y","3T","3TY","3A","3AY","3@","3@Y","4","4Y","4T","4TY","4A","4AY","4@","4@Y"]
rel_2_stand=dict(zip(relevant_filing_types, stand_filing_types))
stand_2_rel=dict(zip(stand_filing_types, relevant_filing_types))

quarter=2
year=2023
rem_all=pd.DataFrame() #empty dataframe to start, will add to it
filing_all=pd.DataFrame() #empty dataframe to start, will add to it
for suff in filing_suffixes:
    filing=clean_cols(pd.read_csv("../../lobbying-local-data/LDA_data/Filings_"+str(year)+"/filings_"+str(year)+"_"+stand_2_rel[str(quarter)+suff]+".csv")).drop_duplicates(subset=['reg_id','cl_id'],keep='last') #loading in filings, dropping duplicates keeping most recent
    filing_am=clean_cols(pd.read_csv("../../lobbying-local-data/LDA_data/Filings_"+str(year)+"/filings_"+str(year)+"_"+stand_2_rel[str(quarter)+fil_2_am[suff]]+".csv")).drop_duplicates(subset=['reg_id','cl_id'],keep='last') # same for amendments
    rem=pd.concat([filing_am, filing, filing]).drop_duplicates(subset=['reg_id','cl_id'],keep=False) #gives all amendments with no matching filing, corresponds to rem_A, etc in discussion above
    filing_am_matched=pd.concat([filing_am,rem]).drop_duplicates(subset=['reg_id','cl_id'],keep=False) #gives all matched amendments
    filing=pd.concat([filing, filing_am_matched]).drop_duplicates(subset=['reg_id','cl_id'],keep='last') #updates filing with the matched amendments.
    rem_all=pd.concat([rem_all, rem]).drop_duplicates(subset=['reg_id','cl_id'],keep='first') #adds rem to rem_all, tossing repeats
    filing_all=pd.concat([filing_all, filing]).drop_duplicates(subset=['reg_id','cl_id'],keep='first')
filing_all.sort_values('dt_posted',axis=0)
rem_all.sort_values('dt_posted',axis=0)
final_filing=pd.concat([filing_all, rem_all]).drop_duplicates(subset=['reg_id','cl_id'],keep='last').sort_values('dt_posted',axis=0) #this replaces entries in filing_all that match something in rem_all, and appends the leftovers (true_rem) to the dataframe

In [143]:
final_filing.head()

,registrant,client,income,expenses,issue_codes,entities_lobbied,lobbying_description,foreign_entities,affiliated_organizations,JACK_conviction,filing_uuid,filing_type,expenses_method,dt_posted,reg_id,cl_id
0,"SARAH BLACKWOOD GOVERNMENT RELATIONS, LLC",NOVELIS INC.,20000.0,0.0,[],[],[],[],[],False,3bbf137f-98d2-47d5-ad28-8c39affab1e0,2TY,0,2023-04-03T16:23:26,401103935,200384
0,ROSETTA GROUP LLC,CORMAC GROUP ON BEHALF OF NOVA SOUTHEASTERN UN...,10000.0,0.0,[EDU],"[HOUSE OF REPRESENTATIVES, SENATE]",[Labor HHS appropriations],[],[],False,6f0fec31-f304-4e34-8e1e-350cdbcb1bd1,2T,0,2023-04-11T16:40:13,401106009,210139
1,"PARDES BIOSCIENCES, INC.","PARDES BIOSCIENCES, INC.",0.0,120000.0,"[MED, MMM, PHA, HCR]",[Centers For Medicare and Medicaid Services (C...,"[NIH funding for clinical trials, Medicare cov...",[],[],False,7b178cfd-76ff-4ab0-bfa0-f3bee1a34931,2T,A,2023-04-12T10:26:43,401106493,45950
0,"CONTINENTAL STRATEGY, LLC",TETHER OPERATIONS LIMITED,150000.0,0.0,[BAN],[],[Legislation related to stablecoins],[],[],False,e910f68a-0285-4caf-8408-b32716fe7070,Q2,0,2023-04-17T13:52:00,401106285,54177
1,EDENRED NORTH AMERICA INC.,EDENRED NORTH AMERICA INC.,0.0,0.0,[],[],[],[],[],False,34e52f19-185e-4b48-b39e-473d6e6ea34c,2TY,A,2023-04-17T17:15:15,401106600,51333


Let's make this a function.

In [65]:
def update_filings(quarter, year):
    filing_suffixes=['','Y','T','TY']
    amendment_suffixes=['A','AY','@', '@Y']
    fil_2_am=dict(zip(filing_suffixes,amendment_suffixes))
    relevant_filing_types=["Q1","Q1Y","1T","1TY","1A","1AY","1@","1@Y","Q2","Q2Y","2T","2TY","2A","2AY","2@","2@Y","Q3","Q3Y","3T","3TY","3A","3AY","3@","3@Y","Q4","Q4Y","4T","4TY","4A","4AY","4@","4@Y"]
    stand_filing_types=["1","1Y","1T","1TY","1A","1AY","1@","1@Y","2","2Y","2T","2TY","2A","2AY","2@","2@Y","3","3Y","3T","3TY","3A","3AY","3@","3@Y","4","4Y","4T","4TY","4A","4AY","4@","4@Y"]
    rel_2_stand=dict(zip(relevant_filing_types, stand_filing_types))
    stand_2_rel=dict(zip(stand_filing_types, relevant_filing_types))

    rem_all=pd.DataFrame() #empty dataframe to start, will add to it
    filing_all=pd.DataFrame() #empty dataframe to start, will add to it
    for suff in filing_suffixes:
        filing=clean_cols(pd.read_csv("../../lobbying-local-data/LDA_data/Filings_"+str(year)+"/filings_"+str(year)+"_"+stand_2_rel[str(quarter)+suff]+".csv")).drop_duplicates(subset=['reg_id','cl_id'],keep='last') #loading in filings, dropping duplicates keeping most recent
        filing_am=clean_cols(pd.read_csv("../../lobbying-local-data/LDA_data/Filings_"+str(year)+"/filings_"+str(year)+"_"+stand_2_rel[str(quarter)+fil_2_am[suff]]+".csv")).drop_duplicates(subset=['reg_id','cl_id'],keep='last') # same for amendments
        rem=pd.concat([filing_am, filing, filing]).drop_duplicates(subset=['reg_id','cl_id'],keep=False) #gives all amendments with no matching filing, corresponds to rem_A, etc in discussion above
        filing_am_matched=pd.concat([filing_am,rem]).drop_duplicates(subset=['reg_id','cl_id'],keep=False) #gives all matched amendments
        filing=pd.concat([filing, filing_am_matched]).drop_duplicates(subset=['reg_id','cl_id'],keep='last') #updates filing with the matched amendments.
        rem_all=pd.concat([rem_all, rem]).drop_duplicates(subset=['reg_id','cl_id'],keep='first') #adds rem to rem_all, tossing repeats
        filing_all=pd.concat([filing_all, filing]).drop_duplicates(subset=['reg_id','cl_id'],keep='first')
    filing_all.sort_values('dt_posted',axis=0)
    rem_all.sort_values('dt_posted',axis=0)
    updated_filings=pd.concat([filing_all, rem_all]).drop_duplicates(subset=['reg_id','cl_id'],keep='last').sort_values('dt_posted',axis=0) #this replaces entries in filing_all that match something in rem_all, and appends the leftovers (true_rem) to the dataframe
    updated_filings=updated_filings.drop(labels=['dt_posted'], axis=1)
    return updated_filings

## (5) Dealing with `incomes` and `expenses`.

We now have to figure out what to do when we are unsure about whether the filing should have `income` or `expenses`. First, we repeat a sanity check.

In [9]:
final_filing.loc[(final_filing.income!=0) & (final_filing.expenses!=0)]

,registrant,client,income,expenses,issue_codes,entities_lobbied,lobbying_description,foreign_entities,affiliated_organizations,JACK_conviction,filing_uuid,filing_type,expenses_method,dt_posted,reg_id,cl_id


We are actually worried about the other extreme - when both `income` and `expenses` are not reported.

In [10]:
final_filing.loc[(final_filing.income==0) & (final_filing.expenses==0)]

,registrant,client,income,expenses,issue_codes,entities_lobbied,lobbying_description,foreign_entities,affiliated_organizations,JACK_conviction,filing_uuid,filing_type,expenses_method,dt_posted,reg_id,cl_id
1,EDENRED NORTH AMERICA INC.,EDENRED NORTH AMERICA INC.,0.0,0.0,[],[],[],[],[],False,34e52f19-185e-4b48-b39e-473d6e6ea34c,2TY,A,2023-04-17 17:15:15,401106600,51333
2,BLUEBIRD STRATEGIES,PEAR THERAPEUTICS,0.0,0.0,[],[],[],[],[],False,fc8fc80e-7181-4a8d-94ee-dedf5f403dc3,2TY,0,2023-04-18 10:05:28,401105447,203721
3,CAPITAL HEALTH SYSTEM,CAPITAL HEALTH SYSTEM,0.0,0.0,[],[],[],[],[],False,dd4fdacb-85b1-466a-ae4e-cc0a2e3f05a4,2TY,A,2023-04-21 10:59:48,401104159,200871
4,"COMMONWEALTH STRATEGIC PARTNERS, LLC",SHIPT,0.0,0.0,[],[],[],[],[],False,55995714-96bf-4350-85cd-ead081776b03,2TY,0,2023-04-27 17:04:21,401103486,210383
5,"COMMONWEALTH STRATEGIC PARTNERS, LLC",PENNSYLVANIA ASSISTED LIVING ASSOCIATION (PALA),0.0,0.0,[],[],[],[],[],False,292d5082-fe11-4bc2-83f9-020816f4c92e,2TY,0,2023-04-27 17:05:42,401103486,205253
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2383,JON THOMAS CONSULTING,LETS GO BOYS AND GIRLS,0.0,0.0,[],[],[],[],[],False,9049ed18-0cb2-49ef-89c9-8fc4bbacb56e,Q2Y,0,2024-02-12 11:39:35,401105994,209197
61,INTERNATIONAL ASSOCATION OF DRILLING CONTRACTORS,INTERNATIONAL ASSOCIATION OF DRILLING CONTRACTORS,0.0,0.0,[],[],[],[],[],False,deb50ef4-9275-43f7-958d-5bf3bad55935,2AY,0,2024-02-28 16:17:43,19742,126399
2384,"TRADEWINS, LLC",HOGAN LOVELLS US LLP (ON BEHALF OF BECTON DICK...,0.0,0.0,[],[],[],[],[],False,380e73ab-c303-4f77-9547-a7a0415e8587,Q2Y,0,2024-02-29 19:04:31,294952,57721
2385,"MCALLISTER & QUINN, LLC",LYCOMING COLLEGE,0.0,0.0,[],[],[],[],[],False,d6bce7fb-0c66-400c-a283-d86c82bb0ff4,Q2Y,0,2024-03-01 17:16:05,292573,57305


We can try to identify which of these are `expenses` by looking at when the registrant is the client. For this, we need to remove junk from the registrant and client columns.

In [11]:
def strip_stop(sentence):
    stop_words = set(stopwords.words('english')+list(punctuation))
    word_tokens = word_tokenize(sentence) # converts the words in word_tokens to lower case and then checks whether they are present in stop_words or not
    filtered_sentence = [w.lower() for w in word_tokens if not w.lower() in stop_words] #with lower case conversion
    filtered_string=""
    for w in filtered_sentence:
        filtered_string=filtered_string+w
    return filtered_string

In [45]:
reg_stripped=[strip_stop(final_filing.iloc[j]['registrant']) for j in range(len(final_filing))]
cl_stripped=[strip_stop(final_filing.iloc[j]['client']) for j in range(len(final_filing))]
final_filing['reg_stripped']=reg_stripped
final_filing['cl_stripped']=cl_stripped


In [13]:
name_and_method=final_filing.loc[(final_filing.reg_stripped == final_filing.cl_stripped) & (final_filing.expenses_method !=0)]
method=final_filing.loc[ (final_filing.expenses_method !=0)]
name=final_filing.loc[(final_filing.reg_stripped == final_filing.cl_stripped)]

In [19]:
name_not_method=pd.concat([name, name_and_method]).drop_duplicates(subset=['reg_id','cl_id'],keep=False)
method_not_name=pd.concat([method, name_and_method]).drop_duplicates(subset=['reg_id','cl_id'],keep=False)

In [22]:
name_not_method

,registrant,client,income,expenses,issue_codes,entities_lobbied,lobbying_description,foreign_entities,affiliated_organizations,JACK_conviction,filing_uuid,filing_type,expenses_method,dt_posted,reg_id,cl_id,reg_stripped,cl_stripped
6,"HOME INSTEAD, INC.","HOME INSTEAD, INC.",75000.0,0.0,[HCR],"[HOUSE OF REPRESENTATIVES, SENATE]",[To amend the Internal Revenue Code of 1986 to...,[],[],False,02e448db-0113-4fd5-90f9-5c4f0f22b3a2,Q2,0,2023-05-25 15:12:30,400495440,191832,homeinsteadinc,homeinsteadinc
95,CLERGY FINANCIAL STRATEGY AND TAX PLANNING INC,CLERGY FINANCIAL STRATEGY AND TAX PLANNING INC,0.0,0.0,[],[],[],[],[],False,d5296a84-4b2d-4042-9260-df26e8c32b81,Q2Y,0,2023-07-05 15:15:56,400689053,194255,clergyfinancialstrategytaxplanninginc,clergyfinancialstrategytaxplanninginc
589,ENDOCRINE SOCIETY,ENDOCRINE SOCIETY,0.0,0.0,[HCR],[Centers For Disease Control & Prevention (CDC...,[healthcare],[],[],False,d187c562-d2a4-4179-bcd3-0a472c5dde7d,Q2,0,2023-07-06 08:50:46,13574,118991,endocrinesociety,endocrinesociety
843,CHAMBER OF SHIPPING OF AMERICA,CHAMBER OF SHIPPING OF AMERICA,0.0,0.0,[MAR],[Centers For Disease Control & Prevention (CDC...,[Ocean Shipping Antitrust Enforcement Act - HR...,[],[],False,263efa72-5fcd-4f43-9941-1089bf3bb0a5,Q2,0,2023-07-07 16:16:50,38767,146968,chambershippingamerica,chambershippingamerica
14,"SLEEP, INC.","SLEEP, INC.",0.0,0.0,[HCR],"[HOUSE OF REPRESENTATIVES, Health & Human Serv...",[Adolescent sleep health; sleep health equity;...,[],[],False,86be26d0-1b79-4095-98cf-82558042e1cb,2A,0,2023-07-10 08:51:35,401104032,200657,sleepinc,sleepinc
1068,PROTON THERAPY CONSORTIUM FKA NATIONAL ASSOCIA...,PROTON THERAPY CONSORTIUM FKA NATIONAL ASSOCIA...,0.0,0.0,"[INS, HCR, BUD, MMM]",[Centers For Medicare and Medicaid Services (C...,[Address concerns regarding prior authorizatio...,[],[],False,66c6b758-9e40-44e9-b9a7-367c2b61ae36,Q2,0,2023-07-10 09:56:29,401106232,212568,protontherapyconsortiumfkanationalassociationp...,protontherapyconsortiumfkanationalassociationp...
177,CENTER FOR ARMS CONTROL & NON-PROLIFERATION,CENTER FOR ARMS CONTROL & NON-PROLIFERATION,0.0,0.0,[],[],[],[],[],False,fb076987-4ddd-49f5-bfa1-323433a41e61,Q2Y,0,2023-07-10 11:30:37,49351,156188,centerarmscontrolnon-proliferation,centerarmscontrolnon-proliferation
1105,COUNCIL FOR A LIVABLE WORLD,COUNCIL FOR A LIVABLE WORLD,40000.0,0.0,"[DEF, FOR, BUD]","[HOUSE OF REPRESENTATIVES, SENATE]",[Overall budget including nuclear weapons and ...,[],[],False,944305f7-03eb-4550-8b1a-7d2480aa8ab3,Q2,0,2023-07-10 11:38:00,10970,114910,councillivableworld,councillivableworld
1116,AMERICAN SOCIETY FOR BIOCHEMISTRY AND MOLECULA...,AMERICAN SOCIETY FOR BIOCHEMISTRY AND MOLECULA...,5000.0,0.0,"[BUD, SCI]","[HOUSE OF REPRESENTATIVES, SENATE]","[Increasing budget for NIH, NSF and Department...",[],[],False,a9b6d7c0-54ad-41b3-8be1-dcef8d7342e9,Q2,0,2023-07-10 12:36:54,3344,103654,americansocietybiochemistrymolecularbiology,americansocietybiochemistrymolecularbiology
215,JUVENILE PRODUCTS MANUFACTURERS ASSOCIATION,JUVENILE PRODUCTS MANUFACTURERS ASSOCIATION,0.0,0.0,[],[],[],[],[],False,83b6e842-b36e-4d93-85d0-e027a4b5472d,Q2Y,0,2023-07-11 09:13:21,401105513,203783,juvenileproductsmanufacturersassociation,juvenileproductsmanufacturersassociation


In [27]:
name_not_method.loc[name_not_method['expenses']!=0]

,registrant,client,income,expenses,issue_codes,entities_lobbied,lobbying_description,foreign_entities,affiliated_organizations,JACK_conviction,filing_uuid,filing_type,expenses_method,dt_posted,reg_id,cl_id,reg_stripped,cl_stripped


These `name_not_method`-filings all appear to be `expense`-type filings, but with errors in paperwork. It looks like they entered under `income` sometimes when they should have entered under `expenses`.

In [25]:
method_not_name.sample(10)

,registrant,client,income,expenses,issue_codes,entities_lobbied,lobbying_description,foreign_entities,affiliated_organizations,JACK_conviction,filing_uuid,filing_type,expenses_method,dt_posted,reg_id,cl_id,reg_stripped,cl_stripped
17168,"NATIONAL COOPERATIVE BANK, N.A. (FORMERLY NCB,...","NCB, FSB",0.0,35000.0,[SMB],"[HOUSE OF REPRESENTATIVES, SENATE]",[Cooperative business eligibility for SBA lend...,[],[],False,568a267a-deac-43e3-86c2-8595ac020b51,Q2,A,2023-07-31 10:31:28,401103208,198156,nationalcooperativebankn.aformerlyncbfsb,ncbfsb
9562,"SOUNDEXCHANGE, INC.",SOUNDEXCHANGE,0.0,240000.0,[CPT],"[HOUSE OF REPRESENTATIVES, SENATE]",[Music copyright issues; American Music Fairne...,[],[],False,75c2f750-ad62-4cab-926a-372abebdc055,Q2,A,2023-07-19 21:23:51,319734,186503,soundexchangeinc,soundexchange
11915,"KOCH GOVERNMENT AFFAIRS, LLC (FORMERLY KNOWN A...","KOCH GOVERNMENT AFFAIRS, LLC (FORMERLY KNOWN A...",0.0,2020000.0,"[TEC, AGR, BNK, UTI, CHM, TRD, TRA, TAX, LAW, ...","[HOUSE OF REPRESENTATIVES, SENATE]",[General issues relating to spectrum reauthori...,[],[],False,41d625cd-d784-4e78-a94a-c05021934ce6,Q2,C,2023-07-20 11:54:36,400390816,189987,kochgovernmentaffairsllcformerlyknownkochcompa...,kochgovernmentaffairsllcformerlyknownkochcompa...
1955,STEPHEN BASSETT,PARADIGM RESEARCH GROUP (FORMALLY KNOWN AS STE...,0.0,0.0,[],[],[],[],[],False,2a576a7d-6cdb-4629-bbfc-ad7f1a96def1,Q2Y,A,2023-07-24 12:43:50,30713,136018,stephenbassett,paradigmresearchgroupformallyknownstephenbassett
5702,LIVE OAK BANKING COMPANY (FORMERLY KNOWN AS LI...,LIVE OAK BANK,0.0,110000.0,"[AGR, SMB, BUD, BAN]","[Agriculture, Dept of (USDA), Federal Deposit ...","[- Issues related to 2023 Farm bill, including...",[],[],False,c125c8fa-b6c4-4822-9bfb-bf235f4186de,Q2,A,2023-07-18 15:18:18,401104559,202042,liveoakbankingcompanyformerlyknownliveoakbank,liveoakbank
16589,ADISA - ALTERNATIVE & DIRECT INVESTMENT,ALTERNATIVE & DIRECT INVESTMENT SECURITIES ASS...,0.0,114133.0,"[ENV, TAX, ACC, BUD]",[HOUSE OF REPRESENTATIVES],"[ESG report requirement, Fiduciary rule, Accre...",[],[],False,b0467168-68d3-4610-96b7-49b48e0b7d8e,Q2,A,2023-07-21 10:23:33,401104118,200794,adisaalternativedirectinvestment,alternativedirectinvestmentsecuritiesassociation
8768,"ASTELLAS GENE THERAPIES, INC. A DELAWARE CORPO...","ASTELLAS GENE THERAPIES, INC. A DELAWARE CORPO...",0.0,90000.0,"[HCR, MMM]","[HOUSE OF REPRESENTATIVES, SENATE]","[Value of Prescription Drugs, Drug pricing, Bi...",[],[],False,8ac2a5c1-eaf5-43c3-97d3-e2a217dbf60e,Q2,A,2023-07-19 16:47:34,401105738,206554,astellasgenetherapiesinc.delawarecorporationfk...,astellasgenetherapiesinc.delawarecorporationfk...
587,NATIONAL EMPLOYMENT LAWYERS ASSOCIATION,NATIONAL EMPLOYMENT LAWYERS ASSOCIATION (NELA),0.0,20000.0,"[CIV, LBR]",[Equal Employment Opportunity Commission (EEOC...,[trengthening of laws against sexual and workp...,[],[],False,7d2233ef-9666-449b-88dc-5fc8dfb21f0a,2A,A,2024-01-17 15:29:45,312736,184005,nationalemploymentlawyersassociation,nationalemploymentlawyersassociationnela
2108,A.C.E. CONSULTING DBA A.C.E. CONSTRUCTION CONS...,A.C.E. CONSULTING,0.0,0.0,[],[],[],[],[],False,c6fcba72-9247-4812-b23b-e44c73249364,Q2Y,A,2023-08-06 21:16:00,400656972,193898,a.c.econsultingdbaa.c.econstructionconsulting,a.c.econsulting
14368,"AT&T SERVICES, INC. AND ITS AFFILIATES",AT&T SERVICES INC AND ITS AFFILIATES,0.0,2780000.0,"[TRD, TAX, TEC]","[Commerce, Dept of (DOC), Federal Aviation Adm...","[issues related to the enforcement of USMCA, S...",[],[],False,9b3d50f4-475c-40a6-8f0a-cf760e5d50b1,Q2,A,2023-07-20 16:02:59,34455,142919,servicesinc.affiliates,servicesincaffiliates


These `method_not_name` entries all appear to be true `expense`-type filings.

So we should look for all filings which either have matching registrant-client entires, or have non-zero entries in `expenses_method`. These are all of the true `expense`-type filings.

In [55]:
expense_type=final_filing.loc[(final_filing.reg_stripped == final_filing.cl_stripped) | (final_filing.expenses_method !=0)].copy()
income_type=pd.concat([final_filing, expense_type]).drop_duplicates(subset=['reg_id','cl_id'], keep=False)

In [56]:
exp_sp=expense_type['expenses']+expense_type['income']
expense_type['expenses']=exp_sp
expense_type['income']=float(0)
inc_sp=income_type['expenses']+income_type['income']
income_type['income']=inc_sp
income_type['expenses']=float(0)

In [57]:
expense_type.loc[expense_type.expenses==0, 'expenses']=float(1)
income_type.loc[income_type.income==0, 'income']=float(1)

In [58]:
expense_type

,registrant,client,income,expenses,issue_codes,entities_lobbied,lobbying_description,foreign_entities,affiliated_organizations,JACK_conviction,filing_uuid,filing_type,expenses_method,dt_posted,reg_id,cl_id,reg_stripped,cl_stripped
1,"PARDES BIOSCIENCES, INC.","PARDES BIOSCIENCES, INC.",0.0,120000.0,"[MED, MMM, PHA, HCR]",[Centers For Medicare and Medicaid Services (C...,"[NIH funding for clinical trials, Medicare cov...",[],[],False,7b178cfd-76ff-4ab0-bfa0-f3bee1a34931,2T,A,2023-04-12 10:26:43,401106493,45950,pardesbiosciencesinc,pardesbiosciencesinc
1,EDENRED NORTH AMERICA INC.,EDENRED NORTH AMERICA INC.,0.0,1.0,[],[],[],[],[],False,34e52f19-185e-4b48-b39e-473d6e6ea34c,2TY,A,2023-04-17 17:15:15,401106600,51333,edenrednorthamericainc,edenrednorthamericainc
3,CAPITAL HEALTH SYSTEM,CAPITAL HEALTH SYSTEM,0.0,1.0,[],[],[],[],[],False,dd4fdacb-85b1-466a-ae4e-cc0a2e3f05a4,2TY,A,2023-04-21 10:59:48,401104159,200871,capitalhealthsystem,capitalhealthsystem
2,"CROSSFIT, LLC","CROSSFIT, LLC",0.0,40000.0,[HCR],"[HOUSE OF REPRESENTATIVES, SENATE]",[Advocating for fitness facilities to be desig...,[],[],False,be598468-d4d2-4443-9a92-fb18aea942f3,2T,A,2023-04-27 17:18:55,401105803,207138,crossfitllc,crossfitllc
14,INTERNATIONAL SLEEP PRODUCTS ASSOCIATION,INTERNATIONAL SLEEP PRODUCTS ASSOCIATION,0.0,1.0,[],[],[],[],[],False,da0e5e86-2828-4dc9-b26b-39a644750a03,2TY,A,2023-05-10 10:25:18,46780,154308,internationalsleepproductsassociation,internationalsleepproductsassociation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2380,ALECTA STRATEGIES LLC,ALECTA STRATEGIES LLC,0.0,1.0,[],[],[],[],[],False,7d410540-5f4b-4e46-b5e7-4d260cd74cf2,Q2Y,A,2024-02-05 13:52:13,401105576,205735,alectastrategiesllc,alectastrategiesllc
18351,THE AMERICAN COUNCIL OF ENGINEERING COMPANIES,AMERICAN COUNCIL OF ENGINEERING COMPANIES,0.0,987181.0,"[ENV, LBR, FOR, FIN, TRA, TOR, TAX, ROD, IMM, ...","[Army, Dept of (Corps of Engineers), Commerce,...",[Funding for Superfund and other hazardous was...,[],[],False,f629416e-cd3b-4bb0-95f0-1539e7940bdc,Q2,C,2024-02-08 15:15:08,2046,102678,americancouncilengineeringcompanies,americancouncilengineeringcompanies
600,BIOGEN,BIOGEN,0.0,830000.0,"[CPT, BUD, TRD, TAX, PHA, MMM, HCR]",[Centers For Medicare and Medicaid Services (C...,[Support stronger IP and exclusivity protectio...,[],[],False,5332bece-7d29-4dcf-94c4-8d2afd1bc6da,2A,A,2024-02-13 16:45:37,48725,155476,biogen,biogen
601,THE AMERICAN PUBLIC GAS ASSOCIATION,AMERICAN PUBLIC GAS ASSOCIATION,0.0,10000.0,[FUE],"[HOUSE OF REPRESENTATIVES, SENATE]",[Amend Section 5 of the Natural Gas Act and pr...,[],[],False,0cdbce58-8498-47a9-ba8e-11a179bfd30a,2A,A,2024-02-15 09:25:13,74077,167305,americanpublicgasassociation,americanpublicgasassociation


In [51]:
income_type

,registrant,client,income,expenses,issue_codes,entities_lobbied,lobbying_description,foreign_entities,affiliated_organizations,JACK_conviction,filing_uuid,filing_type,expenses_method,dt_posted,reg_id,cl_id,reg_stripped,cl_stripped
0,"SARAH BLACKWOOD GOVERNMENT RELATIONS, LLC",NOVELIS INC.,20000.0,1.0,[],[],[],[],[],False,3bbf137f-98d2-47d5-ad28-8c39affab1e0,2TY,0,2023-04-03 16:23:26,401103935,200384,sarahblackwoodgovernmentrelationsllc,novelisinc
0,ROSETTA GROUP LLC,CORMAC GROUP ON BEHALF OF NOVA SOUTHEASTERN UN...,10000.0,1.0,[EDU],"[HOUSE OF REPRESENTATIVES, SENATE]",[Labor HHS appropriations],[],[],False,6f0fec31-f304-4e34-8e1e-350cdbcb1bd1,2T,0,2023-04-11 16:40:13,401106009,210139,rosettagroupllc,cormacgroupbehalfnovasoutheasternuniversity
0,"CONTINENTAL STRATEGY, LLC",TETHER OPERATIONS LIMITED,150000.0,1.0,[BAN],[],[Legislation related to stablecoins],[],[],False,e910f68a-0285-4caf-8408-b32716fe7070,Q2,0,2023-04-17 13:52:00,401106285,54177,continentalstrategyllc,tetheroperationslimited
2,BLUEBIRD STRATEGIES,PEAR THERAPEUTICS,0.0,1.0,[],[],[],[],[],False,fc8fc80e-7181-4a8d-94ee-dedf5f403dc3,2TY,0,2023-04-18 10:05:28,401105447,203721,bluebirdstrategies,peartherapeutics
4,"COMMONWEALTH STRATEGIC PARTNERS, LLC",SHIPT,0.0,1.0,[],[],[],[],[],False,55995714-96bf-4350-85cd-ead081776b03,2TY,0,2023-04-27 17:04:21,401103486,210383,commonwealthstrategicpartnersllc,shipt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,TORREY ADVISORY GROUP (FORMERLY MICHAEL TORREY...,ILLINOIS SOYBEAN ASSOCIATION,70000.0,1.0,"[TRA, TRD, AGR]","[Agriculture, Dept of (USDA), HOUSE OF REPRESE...",[Funding for locks and dams. Issues relating t...,[],[],False,58676501-546c-4fd8-9773-88aaa9f53455,2A,0,2024-03-01 13:39:19,297033,179527,torreyadvisorygroupformerlymichaeltorreyassoci...,illinoissoybeanassociation
607,TORREY ADVISORY GROUP (FORMERLY MICHAEL TORREY...,NATURAL RESOURCES DEFENSE COUNCIL AND AFFILIATES,60000.0,1.0,"[FOO, AGR]","[Agriculture, Dept of (USDA), HOUSE OF REPRESE...","[Food Date Labeling., Issues in Farm Bill rela...",[],[],False,e1fe6ef4-941a-4521-bf31-7e60b2ee2f9f,2A,0,2024-03-01 13:51:57,297033,54875,torreyadvisorygroupformerlymichaeltorreyassoci...,naturalresourcesdefensecouncilaffiliates
2385,"MCALLISTER & QUINN, LLC",LYCOMING COLLEGE,0.0,1.0,[],[],[],[],[],False,d6bce7fb-0c66-400c-a283-d86c82bb0ff4,Q2Y,0,2024-03-01 17:16:05,292573,57305,mcallisterquinnllc,lycomingcollege
2386,TERRAPIN STRATEGY INC.,BELEN AEROSPACE,0.0,1.0,[],[],[],[],[],False,6f65e201-20e8-4c99-a839-c2db40874af0,Q2Y,0,2024-03-04 10:05:30,401107434,57578,terrapinstrategyinc,belenaerospace


In [59]:
ultimate_filing=pd.concat([income_type,expense_type]).sort_values('dt_posted')

In [60]:
ultimate_filing

,registrant,client,income,expenses,issue_codes,entities_lobbied,lobbying_description,foreign_entities,affiliated_organizations,JACK_conviction,filing_uuid,filing_type,expenses_method,dt_posted,reg_id,cl_id,reg_stripped,cl_stripped
0,"SARAH BLACKWOOD GOVERNMENT RELATIONS, LLC",NOVELIS INC.,20000.0,0.0,[],[],[],[],[],False,3bbf137f-98d2-47d5-ad28-8c39affab1e0,2TY,0,2023-04-03 16:23:26,401103935,200384,sarahblackwoodgovernmentrelationsllc,novelisinc
0,ROSETTA GROUP LLC,CORMAC GROUP ON BEHALF OF NOVA SOUTHEASTERN UN...,10000.0,0.0,[EDU],"[HOUSE OF REPRESENTATIVES, SENATE]",[Labor HHS appropriations],[],[],False,6f0fec31-f304-4e34-8e1e-350cdbcb1bd1,2T,0,2023-04-11 16:40:13,401106009,210139,rosettagroupllc,cormacgroupbehalfnovasoutheasternuniversity
1,"PARDES BIOSCIENCES, INC.","PARDES BIOSCIENCES, INC.",0.0,120000.0,"[MED, MMM, PHA, HCR]",[Centers For Medicare and Medicaid Services (C...,"[NIH funding for clinical trials, Medicare cov...",[],[],False,7b178cfd-76ff-4ab0-bfa0-f3bee1a34931,2T,A,2023-04-12 10:26:43,401106493,45950,pardesbiosciencesinc,pardesbiosciencesinc
0,"CONTINENTAL STRATEGY, LLC",TETHER OPERATIONS LIMITED,150000.0,0.0,[BAN],[],[Legislation related to stablecoins],[],[],False,e910f68a-0285-4caf-8408-b32716fe7070,Q2,0,2023-04-17 13:52:00,401106285,54177,continentalstrategyllc,tetheroperationslimited
1,EDENRED NORTH AMERICA INC.,EDENRED NORTH AMERICA INC.,0.0,1.0,[],[],[],[],[],False,34e52f19-185e-4b48-b39e-473d6e6ea34c,2TY,A,2023-04-17 17:15:15,401106600,51333,edenrednorthamericainc,edenrednorthamericainc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
607,TORREY ADVISORY GROUP (FORMERLY MICHAEL TORREY...,NATURAL RESOURCES DEFENSE COUNCIL AND AFFILIATES,60000.0,0.0,"[FOO, AGR]","[Agriculture, Dept of (USDA), HOUSE OF REPRESE...","[Food Date Labeling., Issues in Farm Bill rela...",[],[],False,e1fe6ef4-941a-4521-bf31-7e60b2ee2f9f,2A,0,2024-03-01 13:51:57,297033,54875,torreyadvisorygroupformerlymichaeltorreyassoci...,naturalresourcesdefensecouncilaffiliates
2385,"MCALLISTER & QUINN, LLC",LYCOMING COLLEGE,1.0,0.0,[],[],[],[],[],False,d6bce7fb-0c66-400c-a283-d86c82bb0ff4,Q2Y,0,2024-03-01 17:16:05,292573,57305,mcallisterquinnllc,lycomingcollege
18360,"DISABILITY RIGHTS EDUCATION & DEFENSE, INC.",DISABILITY RIGHTS EDUCATION & DEFENSE INC,0.0,10000.0,"[ECN, CIV, HCR]","[Food & Drug Administration (FDA), HOUSE OF RE...",[Recommendations to improve the Supplemental N...,[],[],False,e8226e0e-1ed4-44c7-9c25-cb605d40d6db,Q2,A,2024-03-03 20:40:18,12345,117064,disabilityrightseducationdefenseinc,disabilityrightseducationdefenseinc
2386,TERRAPIN STRATEGY INC.,BELEN AEROSPACE,1.0,0.0,[],[],[],[],[],False,6f65e201-20e8-4c99-a839-c2db40874af0,Q2Y,0,2024-03-04 10:05:30,401107434,57578,terrapinstrategyinc,belenaerospace


We assemble this into a function.

In [61]:
def strip_stop(sentence):
    stop_words = set(stopwords.words('english')+list(punctuation))
    word_tokens = word_tokenize(sentence) # converts the words in word_tokens to lower case and then checks whether they are present in stop_words or not
    filtered_sentence = [w.lower() for w in word_tokens if not w.lower() in stop_words] #with lower case conversion
    filtered_string=""
    for w in filtered_sentence:
        filtered_string=filtered_string+w
    return filtered_string

def inc_exp(df):
    reg_stripped=[strip_stop(df.iloc[j]['registrant']) for j in range(len(df))]
    cl_stripped=[strip_stop(df.iloc[j]['client']) for j in range(len(df))]
    df['reg_stripped']=reg_stripped
    df['cl_stripped']=cl_stripped
    expense_type=df.loc[(df.reg_stripped == df.cl_stripped) | (df.expenses_method !=0)].copy()
    income_type=pd.concat([df, expense_type]).drop_duplicates(subset=['reg_id','cl_id'], keep=False)
    exp_sp=expense_type['expenses']+expense_type['income']
    expense_type['expenses']=exp_sp
    expense_type['income']=float(0)
    inc_sp=income_type['expenses']+income_type['income']
    income_type['income']=inc_sp
    income_type['expenses']=float(0)
    expense_type.loc[expense_type.expenses==0, 'expenses']=float(1)
    income_type.loc[income_type.income==0, 'income']=float(1)
    ult_df=pd.concat([income_type,expense_type]).sort_values('dt_posted')
    ult_df=ult_df.drop(labels=['reg_stripped','cl_stripped'], axis=1)
    return ult_df
    

## Exporting

We can now clean all of our data and export.

In [ ]:
quarters=range(1,5)
years=range(2013,2025)
for y in years:
    for q in quarters:
        df=update_filings(q,y)
        with inc_exp(df)